<a href="https://colab.research.google.com/github/PraljakReps/UChicago_AIplusScience_code_workshops/blob/main/2023_AI%2BScienceSummerSchool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protein-ligand binding affinity prediction using deep learning 🧪🥼💻

### Develop and train deep learning architectures to process protein and ligand data for predicting binding affinity

Things that we will learn from this notebook:

  * **SCIENCE**: understand protein and ligands (e.g. protein primary sequecne, tertiary structure, molecular ligand graph, SMILES strings for molecules).
  * **SCIENCE**: understand the importance of binding affinity for protein-ligand complexes
  * **AI**: preprocess protein sequence, structure data and molecular graph and ligand SMILES string data.
  * **AI**: develop dataloaders in PyTorch to process input data for training various tasks (e.g. protein sequences, SMILES strings, structure).
  * **AI**: develop, train, and implement feedforward neural networks for binding affinity prediction for single- and multi- modality tasks (i.e. handle only protein sequences or ligand SMILES strings, the combination of the two, and handle sequence and structure information).


## Illustrations of protein-ligand binding.

Here, we show a protein (blue) and ligand (orange) binding into a large complex referred to as protein with binding ligand.
![Cartoon illustration of protein-ligand binding](https://upload.wikimedia.org/wikipedia/commons/e/ee/Proten_ligand_binding.PNG?20170819201138)

Here, we show the tertiary structure of the protein (blue) and ligand (red), where the ligand is binding in the pocket of the protein.
![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/Myoglobin_and_heme.png/300px-Myoglobin_and_heme.png)


One way to understand the binding between the protein and ligand is to measure the dissociation constant $K_d$. The $K_d$ in the context of a protein-ligand complex, quantifies the affinity between the protein and the ligand. In other words, it represents the concentration of ligand at which half of the protein is in the ligand-bound state at equilibrium.

If a protein and ligand interact to form a complex, this can be represented as:

P + L ⇌ PL

where P is the protein, L is the ligand, and PL is the protein-ligand complex.

The Kd is calculated as follows:

Kd = [P][L]/[PL]

Here, [P], [L], and [PL] are the concentrations of the protein, ligand, and complex, respectively, at equilibrium.

**A lower Kd value indicates a higher affinity of the protein for the ligand because less ligand is needed to saturate the protein. Conversely, a higher Kd value suggests a lower affinity, as more ligand is needed to saturate the protein.**

It's important to note that the Kd value is a specific measure of the equilibrium between the bound and unbound states of the protein and ligand, and does not provide information about the kinetics of binding or unbinding. It is also independent of the concentration of protein or ligand, assuming these concentrations are in excess over the Kd.
## **Goal:**
 We aim to obtain data about proteins and ligands, either through their sequence details or 3D structural information. We then plan to utilize deep learning techniques to analyze this data and make predictions about their affinities, in other words, their propensity to bind with each other.




In [ ]:
#@markdown #**Check GPU type** 🕵️
#@markdown ### Factory reset runtime if you don't have the desired GPU.

#@markdown ---




#@markdown V100 = Excellent (*Available only for Colab Pro users*)

#@markdown P100 = Very Good

#@markdown T4 = Good (*preferred*)

#@markdown K80 = Meh

#@markdown P4 = (*Not Recommended*)

#@markdown ---

!nvidia-smi -L

In [ ]:

!pip show mdtraj

In [ ]:
#@title Download packages, data, and libraries... 🏗️
# @markdown This cell will take a little while because it has to download several libraries.

#@markdown ---


# clone remote repo that is specific to this notebook
!git clone https://github.com/PraljakReps/UChicago_AIplusScience_code_workshops.git

# install packages
!pip install -q mdtraj nglview
!pip install rdkit
!pip install deepchem
!pip install biopython
!pip install torch_geometric

# download data....
!wget https://pdbbind.oss-cn-hangzhou.aliyuncs.com/download/PDBbind_v2020_other_PL.tar.gz
# download and prepare datasets...
!mkdir /content/v2020

!tar -xvzf PDBbind_v2020_other_PL.tar.gz -C /content/v2020

# download spreadsheet information
!wget https://pdbbind.oss-cn-hangzhou.aliyuncs.com/download/PDBbind_v2020_plain_text_index.tar.gz
!tar -xvzf PDBbind_v2020_plain_text_index.tar.gz


In [ ]:
#@title Import software ⚙️
#@markdown Define necessary functions.

#@markdown ---
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from rdkit import Chem
import mdtraj as md
from rdkit import Chem, RDLogger
import mdtraj as md
import nglview
from rdkit import Chem
from rdkit.Chem import Draw
import deepchem as dc
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

from IPython.display import display, Image
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
#@title Loading and reading data 📦

import pandas as pd

with open('./index/INDEX_general_PL_data.2020', 'r') as file:
  lines = file.readlines()

data = []
for line in lines[6:]:
    line = line.strip()
    fields = line.split()

    pdb_code = fields[0]
    resolution = fields[1]
    release_year = fields[2]
    binding_data = fields[3]
    affinity_measurement = fields[4]

    data.append([pdb_code, resolution, release_year, binding_data, affinity_measurement])

columns = ["PDB code", "resolution", "release year", "binding data", "exp_measurement"]
df = pd.DataFrame(data, columns=columns)
pdb_list = os.listdir('/content/v2020/v2020-other-PL/') # get the pdb complexes that are found in the v2020-other-PL folder...
df = df[df['PDB code'].isin(pdb_list)] # only use complexes that are found in the v2020-other-PL folder...


# Preprocess data into readable format ⚒️

The data contains experimental measurements in terms of three variables: dissociation constant $K_d$, inhibition constant $K_i$, and half-maximal inhibitory concentration $IC50$. For our study, we will focus only on protein-ligand complexes with corredponing dissociation constnats $K_d$, however, I will define below each variable to understand the importance of the measurements.

- $K_d$ (dissociation constant): This is a measure of the affinity of the protein for its ligand. It is the concentration of ligand at which half of the protein is bound to the ligand at equilibrium. A lower Kd indicates higher affinity (because less ligand is needed to bind half of the protein), while a higher Kd indicates lower affinity.

- $K_i$ (inhibition constant): In the context of enzyme kinetics, this is the measure of the potency of an inhibitor, which is a molecule that binds to an enzyme and decreases its activity. The Ki is the concentration of inhibitor that is needed to reduce the binding of a substrate to an enzyme by half in the presence of a given amount of substrate. Just like the Kd, a lower Ki means that the inhibitor has higher affinity for the enzyme, and it takes less inhibitor to achieve half inhibition.

- $IC50$ (half-maximal inhibitory concentration): This is a measure of the effectiveness of a substance in inhibiting a specific biological or biochemical function. It is the concentration of an inhibitor where the response (or binding) is reduced by half. IC50 is often used in drug discovery to judge the potency of a substance. It's important to note that IC50 is not a direct measure of affinity or dissociation constant but provides a functional measure of inhibitor potency under specific conditions. It can be related to Ki but the relationship depends on the specific conditions of the assay from which it is derived.

If you are interested in further understanding these three variables, feel free to read the following resources:

 - short blog: https://www.sciencesnail.com/science/the-difference-between-ki-kd-ic50-and-ec50-values
 - academic paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3000649/

In [ ]:
# Kd (dissociation Constant) datasheet
Kd_df = df[
    df['exp_measurement'].str.contains('Kd')
].reset_index(drop=True)

# Ki (Inhibition Constant) datasheet
Ki_df = df[
    df['exp_measurement'].str.contains('Ki')
].reset_index(drop=True)

# IC50 datasheet
IC50_df = df[
    df['exp_measurement'].str.contains('IC50')
].reset_index(drop=True)

print('Datasheet corresponding to protein-ligand complexes with (Kd) dissociation constants.')
display(Kd_df.head(3))
print('\n\nDatasheet corresponding to protein-ligand complexes with (Ki) Inhibition constants.')
display(Ki_df.head(3))
print('\n\nDatasheet corresponding to protein-ligand complexes with (IC50) Half-Maximal Inhibitory Concentration.')
display(IC50_df.head(3))


In [ ]:
#@title Function for retrieving ligand and protein structure data paths. 🛠️
def add_molecule_data_paths(df: pd.Series):

  data_path_dict = {
      'ligand_mol_paths': [],
      'ligand_sdf_paths': [],
      'pocket_paths': [],
      'protein_pdb_paths': []
  }
  for pdb_id in tqdm(df['PDB code']):

    pdb_path = f'/content/v2020/v2020-other-PL/{pdb_id}/{pdb_id}_'

    # track data paths
    data_path_dict['ligand_mol_paths'].append( pdb_path + 'ligand.mol2' )
    data_path_dict['ligand_sdf_paths'].append( pdb_path + 'ligand.sdf' )
    data_path_dict['pocket_paths'].append( pdb_path + 'pocket.pdb' )
    data_path_dict['protein_pdb_paths'].append( pdb_path + 'protein.pdb' )

  data_path_df = pd.DataFrame(data_path_dict)

  # return the final appended dataframe
  return pd.concat((df, data_path_df), axis=1)


In [ ]:
' Prepare the dataframe with the path to the data files for the ligand and protein structures.'

# attach the protein, ligand data paths
Kd_df = add_molecule_data_paths(df=Kd_df)
Ki_df = add_molecule_data_paths(df=Ki_df)
IC50_df = add_molecule_data_paths(df=IC50_df)

# Visualize the added columns that point the protein, ligand sequence/structure data paths...
print('Dataframe corresponding to the protein-ligand compex dataframe with (Kd) dissociation constants.')
display(Kd_df.head(3))
print('\n\nDataframe corresponding to the protein-ligand compex dataframe with (Ki) Inhibition constants.')
display(Ki_df.head(3))
print('\n\nDataframe corresponding to the protein-ligand compex dataframe with (IC50) Half-Maximal Inhibitory Concentration.')
display(IC50_df.head(3))


In [ ]:
# Load the dataframes with sequence and ligand molecules
temp_Kd_df = pd.read_csv('/content/UChicago_AIplusScience_code_workshops/data/final_Kd_dataframe.csv')
temp_Ki_df = pd.read_csv('/content/UChicago_AIplusScience_code_workshops/data/final_Ki_dataframe.csv')
temp_IC50_df = pd.read_csv('/content/UChicago_AIplusScience_code_workshops/data/final_IC50_dataframe.csv')


# match the uploaded dataframe and match the rows with the colab env dataframe
# --> processing the final Kd dataframe..
final_Kd_df = pd.merge(Kd_df, temp_Kd_df, on='PDB code', suffixes=('', '_x'))
final_Kd_df = final_Kd_df[final_Kd_df.columns.drop(list(final_Kd_df.filter(regex='_x')))]
final_Kd_df = final_Kd_df.dropna(subset=['ligand_smiles'])

# --> processing the final Ki dataframe..
final_Ki_df = pd.merge(Ki_df, temp_Ki_df, on='PDB code', suffixes=('', '_x'))
final_Ki_df = final_Ki_df[final_Ki_df.columns.drop(list(final_Ki_df.filter(regex='_x')))]
final_Ki_df = final_Ki_df.dropna(subset=['ligand_smiles'])

# --> processing the final IC50 dataframe..
final_IC50_df = pd.merge(IC50_df, temp_IC50_df, on='PDB code', suffixes=('', '_x'))
final_IC50_df = final_Ki_df[final_IC50_df.columns.drop(list(final_IC50_df.filter(regex='_x')))]
final_IC50_df = final_IC50_df.dropna(subset=['ligand_smiles'])


In [ ]:
print('Two new columns, containing a protein sequence and ligand smile sequences.')
display(final_Kd_df[['PDB code', 'protein_sequence', 'ligand_smiles']].head(3))

In [ ]:
#@title Problem: create a function to convert string to value 🚩

#@markdown **Goal** of the function `convert_unit_to_value` is to convert experimennt measurement ($K_d$, $K_i$, $IC50$) into a floating point value which can be used for inference and training.

#@markdown E.g., convert $str(K_d=250mM)$ into $float(250*1e-3)$

#@markdown **Note**: functions from re will be helpful to extracting the numerical value from a given string. We will find this line of code for you as a reference ( re.findall(r'(\d+(?:\.\d+)?)\s*([a-zA-Z]+)', value)[0]), where value is the $str(K_d=250mM)$. Ref: [re.finalall()](https://www.codecademy.com/resources/docs/python/regex/findall).

#@markdown ---

import re

def convert_unit_to_value(value: str) -> float:

  # Define a dictionary of conversion factors for different units
  conversion_factors = {
      'M': 1.0,
      'mM':1e-3,
      'uM':1e-6,
      'nM': 1e-9,
      'pM': 1e-12,
      'fM': 1e-15
  }

  # Extract the numerical value and unit
  numerical_value, unit = re.findall(r'(\d+(?:\.\d+)?)\s*([a-zA-Z]+)', value)[0] # e.g. spit kd=255nM into ('255', 'nM').

  # apply the conversion factor from the dictionary
  converted_value = float(numerical_value) * conversion_factors[unit]
  return converted_value / conversion_factors['uM']

In [ ]:
#@title Convert string-based experimental measurements into floating values.

# convert string-based experimental measurements into
final_Kd_df['Kd_values[uM]'] = [convert_unit_to_value(value) for value in final_Kd_df['exp_measurement']]
final_Ki_df['Ki_values[uM]'] = [convert_unit_to_value(value) for value in final_Ki_df['exp_measurement']]
final_IC50_df['IC50_values[uM]'] = [convert_unit_to_value(value) for value in final_IC50_df['exp_measurement']]


In [ ]:
#@title  Display the dataframe columns that corresponds to the preprocess output measurements

print('Kd measurements after preprocessing...')
display(final_Kd_df['Kd_values[uM]'].head(3))
print('\n\nKi measurements after preprocessing...')
display(final_Ki_df['Ki_values[uM]'].head(3))
print('\n\nIC50 measurements after preprocessing...')
display(final_IC50_df['IC50_values[uM]'].head(3))


In [ ]:
#@title Split Kd data into training/testing splits:
from sklearn.model_selection import train_test_split

# split the Kd dataframe into a train/test split:
train_Kd_df, test_Kd_df = train_test_split(final_Kd_df, test_size=0.2, random_state=42)

# Reset index for both dataframes
train_Kd_df = train_Kd_df.reset_index(drop=True)
test_Kd_df = test_Kd_df.reset_index(drop=True)

print('Training and testing dataset sizes:', train_Kd_df.shape[0], test_Kd_df.shape[0])

In [ ]:
#@title Problem: create a function to normalize experimental data 🚩

#@markdown Goal of the function `normalize_values` is to normalize $K_d$, $K_i$, and $IC50$. Normalization of input data is a common preprocessing step in machine learning (ML). Here are some reasons why we might normalize affinity measurements in ML:

#@markdown - **Easier Optimization**: Some ML algorithms, especially those that use gradient descent for optimization, converge faster when features are on similar scales. For example, in a neural network, having all input features in a similar range allows the network to learn weights more effectively.
#@markdown - **Prevent Numerical Instability**: Some ML algorithms can suffer from numerical instability (e.g., underflows and overflows) when working with very large or very small numbers. Normalization can help mitigate these issues.
#@markdown - **Equal Feature Importance**: If the scales of the features vary greatly, some ML algorithms may implicitly treat features with larger scales as more important. Normalization ensures all features have the same scale, so all features are treated equally (at least in terms of scale).

#@markdown **Problem**: Here, we will want to normalize $K_d$ using the max-min normalization technique and then apply logarithm over binding affinities. (Apply the same technique for $K_i$ and $IC50$, individually).

#@markdown The max-min normalization is the following: $$y_{norm} = \frac{y - min(y)}{max(y) - min(y)}$$ where $y$ is the experimental measurements.

#@markdown  note: when normalizing the test set, use min(y) and max(y) statistics from the training set.

#@markdown ---

def normalize_values(values: np.single, min: float, max: float) -> list:
  return (values - min) / (max-min)

# normalize the training and test experimental measurements based on the training set statistics
train_Kd_df['norm_Kd_values'] = normalize_values(
    values=train_Kd_df['Kd_values[uM]'].values,
    min=min(train_Kd_df['Kd_values[uM]'].values.tolist()),
    max=max(train_Kd_df['Kd_values[uM]'].values.tolist())

)
# we want to take the logarithm of the normalized Kd measurements to reduce the dynamic range
train_Kd_df['log_norm_Kd'] = np.log10(
    train_Kd_df['norm_Kd_values'] + 1e-10
)

# Normalize the test set now...
test_Kd_df['norm_Kd_values'] = normalize_values(
    values=test_Kd_df['Kd_values[uM]'].values,
    min=min(train_Kd_df['Kd_values[uM]'].values.tolist()), # important to note... use the training set statistics
    max=max(train_Kd_df['Kd_values[uM]'].values.tolist())

    )
# we want to take the logarithm of the normalized Kd measurements to reduce the dynamic range
test_Kd_df['log_norm_Kd'] = np.log10(
    test_Kd_df['norm_Kd_values'] + 1e-10
)

#### Problem: can you explain why we are able to take the logarithm of $K_d$, $K_i$, or $IC50$ measurements? 🚩

note: think about the role of units (i.e. M, mM, uM etc).

---

**Answer:** $<$insert your explanation here$>$

---


In [ ]:
#@title Visualize the expected training/testing split distribution in terms of log( norm Kd).
image_path = "./UChicago_AIplusScience_code_workshops/figures/Kd_dataset_split.png"
display(Image(filename=image_path,width=500))

In [ ]:
#@title Plot your normalized data and verify whether it matches the above figure.

#@markdown **Problem: does your plot reproduce the above distributions for training and testing splits?** 🚩


# plot the log normalized kd measurements...
plt.hist(train_Kd_df['log_norm_Kd'].values, bins=20, edgecolor='k', label='Train')
plt.hist(test_Kd_df['log_norm_Kd'].values, bins=20, edgecolor='k', label='Test')
plt.xlabel('log( norm Kd)')
plt.ylabel('Number of protein-ligand complexes')
plt.legend()


## Visualize molecule and protein structure 🔎


In [ ]:
#@title Functions used for visualizing protein structures and ligand graphs 🛠️
def visualize_protein(pdb_path: str) -> None:
  """
  function description: view the protein pdb structure in notebook
  """
  protein_mdtraj = md.load_pdb(pdb_path) # protein PDB complex
  # Get the topology object associated with the protein structure
  topology = protein_mdtraj.topology

  # Retrieve the protein sequence
  protein_sequence = topology.to_fasta()[0]

  view = nglview.show_mdtraj(protein_mdtraj)
  print('Protein sequence:', protein_sequence)
  display(view)  # interactive view outside Colab
  return None

def visualize_ligand(ligand_sdf_path: str) -> None:
    """
    Function description: View the molecular ligand using RDKit
    """

    # Load the SDF file
    suppl = Chem.SDMolSupplier(ligand_sdf_path)

    # Iterate over molecules in the SDF file
    for mol in suppl:
        if mol is not None:  # Check if molecule was read successfully
            # Display the molecule
            print("SMILEs ligand sequence:", Chem.MolToSmiles(mol))
            display(Draw.MolToImage(mol))
        else:
            # If the molecule was not read successfully, raise an error
            raise ValueError("Unable to generate Molecular graph")

    return None

Cool examples of molecular graphs and protein structures are the following:
  - `sample_index=50` and `train_set=True` # example from training set
  - `sample_index=1423` and `train_set=True` # example from training set
  - `sample_index=142` and `train_set=False` # example from test set
  - `sample_index=100` and `train_set=False` # example from test set

  Note: max `sample_index` for training and testing set is `2883` and `721`

In [ ]:
#@title Visualize molcules 🔎

sample_index=1423#@param{type:"integer"}
train_set=True#@param {type:"boolean"} # generate from the training

if train_set:
  visualize_ligand(ligand_sdf_path=train_Kd_df['ligand_sdf_paths'].loc[sample_index])
  visualize_protein(pdb_path=train_Kd_df['protein_pdb_paths'].loc[sample_index])
  print(f"Binding affinity: {train_Kd_df['exp_measurement'].loc[sample_index]}")
else:
  visualize_ligand(ligand_sdf_path=test_Kd_df['ligand_sdf_paths'].loc[sample_index])
  visualize_protein(pdb_path=test_Kd_df['protein_pdb_paths'].loc[sample_index])
  print(f"Binding affinity: {test_Kd_df['exp_measurement'].loc[sample_index]}")



# Problem: questions on molecules: proteins and molecular ligands. 🚩

1.) What are the four structural levels used to describe proteins?


**Answer:** $<$insert your explanation here$>$

---

2.) Can you explain how the data processing differs between primary sequence structure and tertiary structure of a protein?

**Answer:** $<$insert your explanation here$>$

---

3.) Could you list the key biochemical groups into which the 20 naturally occurring amino acids in proteins are divided?

**Answer:** $<$insert your explanation here$>$

---

4.) How do hydrophobic amino acids contribute to the formation of a protein's tertiary structure?

**Answer:** $<$insert your explanation here$>$

---

5.) How does the chemical nature of a molecular ligand affect its interaction with protein targets?

**Answer:** $<$insert your explanation here$>$

---


6.) How does the structural complexity of a ligand influence its biological activity?

**Answer:** $<$insert your explanation here$>$
---

# Discriminative task 💪

# Ligand sequence-based encoder model

Here, we will train a single encoder model (i.e. mutli-layer perceptron) model which takes in inputs of ligand sequences as SMILES strings and outputs continous values that corresponding the log normalized dissociation constants log($\tilde{K_d}$). The variable $\tilde{K_d}$ corresponds to the normalized $K_d$ using max-min normalization. To save time during the workshop, we can skip training the model from scratch and load weights from a previously trained configuration. #TODO: insert illustration of the ligand sequence-based encoder.

* Note: we will load the pretrained protein sequence-based encoder model, but we will train the dual ligand- and protein- sequence based encoders and compare test metric performance.

In [ ]:
#@title Illustration of the Ligand-based sequence only encoder architecture.

#@markdown We will process the input data as SMILES string data and use a simple MLP archiecture to predict real-value regression values $y$ (i.e. log( norm Kd)).

image_path = "./UChicago_AIplusScience_code_workshops/figures/Ligand_sequence_encoder.png"
display(Image(filename=image_path,width=500))

In [ ]:
##@markdown `ligand_smiles_dataset` is needed for process a batch of ligand SMILES strings for training a neural network model in pytorch.

#@markdown Note: For now, we will share a complete Dataset class to implement on the Ligand sequence-based encoder as a reference. It will be helpful when creating your Dataset class when developing and training dual encoder model for processing protein and ligand sequences.

#@markdown Second note: `__getitem__()` processes the data for a given batch during training. Please study the `smiles_tokenizer()` and `__getitem__()` hook for how to process batch data in PyTorch and how to convert SMILES strings into tensors.

#@markdown ---

class ligand_smiles_dataset(Dataset):

  def __init__(self, data: list, output_labels: list, max_seq_len: int):

    self.smiles_data = data # input smiles sequences
    self.output_labels = output_labels # binding affinity (Kd measurements)
    self.max_seq_len = max_seq_len # max sequence length
    self.tokenizer = dc.feat.OneHotFeaturizer() # ligand SMILEs tokenization

  def __len__(self):
    return len(self.smiles_data)

  def smiles_tokenizer(self, smiles_sample):

    # convert string characters to one hot encoded tensors
    features =  torch.Tensor(self.tokenizer.featurize(smiles_sample)[:,0,:])

    # compute the padding length
    pad_len = self.max_seq_len - features.size(0)
    # create a tensor of zeros of shape (pad_len, 35)
    pad_tensor = torch.zeros(pad_len, 35)
    # set the last dimension of each vector in pad tensor to 1 (i.e. padded class)
    pad_tensor[:, -1] = 1
    # concatenate features and pad_tensor along dimension 0
    x_padded = torch.cat((features, pad_tensor), dim=0)

    return x_padded

  def __getitem__(self, idx):

    # prepare input data
    smiles = self.smiles_data[idx]
    x_ligands = self.smiles_tokenizer(smiles_sample=smiles)

    # prepare output/prediction data
    y_true = torch.tensor(self.output_labels[idx])

    return (
        x_ligands,
        y_true
    )




In [ ]:
#@title Prepare the Pytorch dataset and dataloader for training

' set up ligand training dataset with Dataset and Dataloader using pytorch'
train_ligand_dataset = ligand_smiles_dataset(
    data=train_Kd_df['ligand_smiles'].tolist(),
    output_labels=train_Kd_df['log_norm_Kd'].tolist(),
    max_seq_len=max(len(s) for s in train_Kd_df['ligand_smiles'].tolist())
)
train_ligand_dataloader = DataLoader(
        train_ligand_dataset,
        batch_size=512,
        shuffle=True,# shuffle is only 'True' for training sets.
        num_workers=0
)
' set up ligand testing dataset with Dataset and Dataloader using pytorch'

test_ligand_dataset = ligand_smiles_dataset(
    data=test_Kd_df['ligand_smiles'].tolist(),
    output_labels=test_Kd_df['log_norm_Kd'].tolist(),
    max_seq_len=max(len(s) for s in train_Kd_df['ligand_smiles'].tolist())
)
test_ligand_dataloader = DataLoader(
        test_ligand_dataset,
        batch_size=512,
        shuffle=False, # shuffle is only 'True' for training sets.
        num_workers=0
)

In [ ]:
#@title Iterate over an batch example with the dataloader.

#@markdown Patience: One pass through the dataloader for the Ligand might take some time since it requires tokenizing SMILES strings.

#@markdown ---

' here we will run a single pass through the data loader as an example '
train_x_ligands, train_y_true = next(iter(train_ligand_dataloader))
print('Training set: Shape of the ligand smile input and output labels:', list(train_x_ligands.shape), list(train_y_true.shape))

' here we will run a single pass through the data loader as an example '
test_x_ligands, test_y_true = next(iter(test_ligand_dataloader))
print('Testing set: Shape of the ligand smile input and output labels:', list(test_x_ligands.shape), list(test_y_true.shape))


In [ ]:
#@title Simple MLP for regression

#@markdown `simple_MLP` is a neural network function that processes Ligand SMILES strings and predicts log $\tilde{K_d}$ values, i.e., $y$~ $p_{\theta}(y|x_l)$.

#@markdown Note: This network is agnostic to input data, and thus can be used for either Ligand SMILES data or Protein sequence data, assuming that architecture hyperparameters are changed accordingly.


#@markdown ---

class simple_MLP(nn.Module):

  def __init__(self, input_size: int, hidden_size: int, depth: int):

    super(simple_MLP, self).__init__()

    self.layers = nn.ModuleList()

    # input layers
    self.layers.append(nn.Linear(input_size, hidden_size))
    self.layers.append(nn.GELU())

    # hidden layers
    for _ in range(depth - 1):

      self.layers.append(nn.Linear(hidden_size, hidden_size))
      self.layers.append(nn.GELU())

    # output layer
    self.layers.append(nn.Linear(hidden_size, 1))

  # forward pass of neural network y ~ f(x)
  def forward(self, x):
    # Flatten the input tensor
    x = x.view(x.size(0), -1)
    for layer in self.layers:
      x = layer(x)
    y_pred = x.clone() # for pedagogical purposes..
    return y_pred


In [ ]:
#@title Define the ligand model hyperparameters for affinity prediction

#@markdown Note: the hyperparameters are given here.
#@markdown ---

seq_len = train_x_ligands.shape[1] # sequence length
char_tokens = train_x_ligands.shape[-1] # number of smiles character tokens
input_size=seq_len * char_tokens # input dimension to MLP
hidden_size=500 # hyperparameter; control expressive the neural network is...
depth=2 # hyperparameter; control how deep the neural network is...

ligand_model = simple_MLP(input_size=input_size, hidden_size=hidden_size, depth=depth)
ligand_model

In [ ]:
#@title Function for tracking performance metrics 🛠️

from scipy.stats import pearsonr, spearmanr

@torch.no_grad()
def performance_metrics(y_true, y_pred):

    if len(y_true.shape) == 2: # if the shape of tensor is (batch_size, 1), squash the last dim. output: (batch_size,)
      y_true.squeeze(-1)

    if len(y_pred.shape) == 2: # if the shape of tensor is (batch_size, 1), squash the last dim. output: (batch_size,)
      y_pred.squeeze(-1)

    # Move tensors to CPU and convert to numpy
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()

    # Compute Pearson correlation
    pearson_corr, _ = pearsonr(y_true, y_pred)

    # Compute Spearman correlation
    spearman_corr, _ = spearmanr(y_true, y_pred)

    return pearson_corr, spearman_corr

In [ ]:
#@title Function for compute metrics on test set 🛠️
@torch.no_grad()
def test_performance_metrics(model: nn.Module, dataloader: DataLoader):

  model.eval()
  # Define a loss function
  criterion = nn.MSELoss()  # For simple regression prediction, we can use mean_squared_error loss.

  y_true_batch, y_pred_batch = [], []

  for i, (x, y_true) in enumerate(dataloader):

    # Move inputs and labels to the device
    x, y_true = x.to(device), y_true.to(device)

    # Forward propagation
    y_pred = model(x).squeeze(1)

    y_true_batch.append(y_true)
    y_pred_batch.append(y_pred)

  # concatenate batches
  y_true_batch = torch.cat(y_true_batch, dim=0)
  y_pred_batch = torch.cat(y_pred_batch, dim=0)

  # Loss computation
  loss = criterion(y_pred_batch, y_true_batch)

  # get metric performance
  pearson_corr, spearman_rho = performance_metrics(
                y_true=y_true_batch,
                y_pred=y_pred_batch
  )
  print(f"Test metrics | Loss: {loss.item():.3f}, Pearson: {pearson_corr.item():.3f}, Spearman rho: {spearman_rho.item():.3f}")

  return (
      loss.item(),
      pearson_corr.item(),
      spearman_rho.item()
  )



In [ ]:
#@title Train the Ligand sequence-based model 🏋️‍♀️

# Define a loss function
criterion = nn.MSELoss()  # For simple regression prediction, we can use mean_squared_error loss.

# define an optimizer
optimizer = optim.SGD(ligand_model.parameters(), lr=0.01)  # lr corresponds to learning_rate (keep fixed for now)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ligand_model.to(device)

# number of training cycles over the dataset (i.e. epochs)
num_epochs = 1

# track losses and metrics
train_history_dict = {
    'step': [],
    'loss': [],
    'pearson_corr': [],
    'spearman_rho': []
}

test_history_dict = {
    'step': [],
    'loss': [],
    'pearson_corr': [],
    'spearman_rho': []
}

iteration = 0
# Start the training loop
for epoch in range(num_epochs):  # num_epochs is the number of epochs to train for
    for i, (x_ligand, y_true) in enumerate(train_ligand_dataloader ):
        # Move inputs and labels to the device
        x_ligand, y_true = x_ligand.to(device), y_true.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward propagation
        y_pred = ligand_model(x_ligand).squeeze(1)

        # Loss computation
        loss = criterion(y_pred, y_true)

        # Backpropagation
        loss.backward()

        # Weight update
        optimizer.step()
        pearson_corr, spearman_rho = performance_metrics(
                y_true=y_true,
                y_pred=y_pred
        )
        print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_ligand_dataloader)}, Loss: {loss.item():.3f}, Pearson: {pearson_corr.item():.3f}, Spearman rho: {spearman_rho.item():.3f}")


        # log training performance
        train_history_dict['step'].append(iteration)
        iteration+=1
        train_history_dict['loss'].append(loss.item())
        train_history_dict['pearson_corr'].append(pearson_corr.item())
        train_history_dict['spearman_rho'].append(spearman_rho.item())

    # compute performance metrics on the test set
    test_loss, test_pearson_corr, test_spearman_rho = test_performance_metrics(model=model, dataloader=test_ligand_dataloader)
    test_history_dict['step'].append(iteration)
    test_history_dict['loss'].append(test_loss)
    test_history_dict['pearson_corr'].append(test_pearson_corr)
    test_history_dict['spearman_rho'].append(test_spearman_rho)


In [ ]:
#@title Visualize training history 🔎

fig, axes = plt.subplots(1, 3, figsize=(10,3))

axes[0].plot(train_history_dict['step'], train_history_dict['loss'])
axes[0].plot(test_history_dict['step'], test_history_dict['loss'], '.-', markersize=10)
axes[0].set_ylabel('loss')
axes[0].set_xlabel('steps')

axes[1].plot(train_history_dict['step'], train_history_dict['pearson_corr'])
axes[1].plot(test_history_dict['step'], test_history_dict['pearson_corr'], '.-', markersize=10)
axes[1].set_ylabel('loss')
axes[1].set_xlabel('steps')

axes[2].plot(train_history_dict['step'], train_history_dict['spearman_rho'])
axes[2].plot(test_history_dict['step'], test_history_dict['spearman_rho'], '.-', markersize=10)
axes[2].set_ylabel('loss')
axes[2].set_xlabel('steps')

plt.tight_layout()

In [ ]:
#@title #### Problem: can you use the loaded pretrain model to predict $log( norm(K_d))$ and compare against ground truth? 🚩

#@markdown Note: you can compute statistics like pearson correlation or spearmann $\rho$ correlation too.

# Protein sequence-based encoder model

We will follow a similar protocol as the `Ligand sequence-based encoder model`; however, we will slightly have to develop a different $dataset$ $iterator$ since protein preprocessing requires a different approach than ligand preprocessing. We can still use the same MLP architecture, but we will require a different input dimension to accoutn for the protein sequence dataset. The steps that this section will inform us is the following:

  - Create dataset iterator and dataloader for processing protein sequences in batch.
  - Create the model architecture for binding affinity prediction.
  - Create the boiler plate for training the model on data. `Note:` To save time for training dual encoder model, we will load pretrained weights and corresponding results on test set.  
  - Post-process results and compare performance ligand encoder model.


In [ ]:
#@title Illustration of the Protein-based sequence only encoder architecture.

#@markdown We will process the input data as Protein amino acid sequence data and use a simple MLP architecture to predict real-value regression values $y$ (i.e. log( norm Kd)).

image_path = "./UChicago_AIplusScience_code_workshops/figures/Protein_sequence_encoder.png"
display(Image(filename=image_path,width=500))

In [ ]:
#@markdown `protein_sequences_dataset` is needed for process a batch of protein sequences  for training a neural network model in pytorch.

#@markdown Note: For now, we will share a complete Dataset class to implement on the protein sequence-based encoder as a reference. It will be helpful when creating your Dataset class when developing and training dual encoder model for processing protein and ligand sequences.


#@markdown Second note: `__getitem__()` processes the data for a given batch during training.

#@markdown Third note: `protein_tokenizer()` converts the amino acid characters into a tensor which can be processed by machine learning algorithms. Please keep track of the `__getitem__()` and `protein_tokenizer()` as it will help with building a dataset iterator when we need to process protein-ligand data in parallel.



#@markdown ---

class protein_sequences_dataset(Dataset):

  def __init__(self, data: list, output_labels: list, max_seq_len: int):

    self.protein_data = data # input smiles sequences
    self.output_labels = output_labels # binding affinity (Kd measurements)
    self.max_seq_len = max_seq_len # max sequence length
    # Define the tokens to cover all the possible amino acids including special ones
    self.tokens = [
        'A', 'C', 'D', 'E', 'F', 'G', 'H',
        'I', 'K', 'L', 'M', 'N', 'P', 'Q',
        'R', 'S', 'T', 'V', 'W', 'Y', 'U',
        'O', '-'
    ]

    # setup the amino acid tokenizer
    self.tokenizer = {token:ii for ii, token in enumerate(self.tokens)}


  def __len__(self):
    return len(self.protein_data)


  def protein_tokenizer(self, protein_sample):
      # convert amino acid characters into numerical values
      num_protein_sample = [self.tokenizer[aa_char] for aa_char in protein_sample]
      # one hot encoding
      one_hot = F.one_hot(torch.tensor(num_protein_sample), num_classes=len(self.tokens))
      # compute the padding length
      pad_len = self.max_seq_len - one_hot.size(0)
      # create a tensor of zeros of shape (pad_len, len(self.tokens))
      pad_tensor = torch.zeros(pad_len, len(self.tokens)).long()
      # set the padding tensor with the 'PAD' index
      pad_tensor[:, self.tokenizer['-']] = 1
      # concatenate features and pad_tensor along dimension 0
      x_padded = torch.cat((one_hot, pad_tensor), dim=0)
      return x_padded

  def __getitem__(self, idx):

    # prepare input data
    protein_sample = self.protein_data[idx]
    x_proteins = self.protein_tokenizer(protein_sample=protein_sample)

    # prepare output/prediction data
    y_true = torch.tensor(self.output_labels[idx])

    return (
        x_proteins.float(),
        y_true
    )




In [ ]:
#@title Create and define Pytorch dataset iterator and dataloader


' set up protein training dataset with Dataset and Dataloader using pytorch'
train_protein_dataset = protein_sequences_dataset(
    data=train_Kd_df['protein_sequence'].tolist(),
    output_labels=train_Kd_df['log_norm_Kd'].tolist(),
    max_seq_len=max(len(s) for s in train_Kd_df['protein_sequence'].tolist())
)
train_protein_dataloader = DataLoader(
        train_protein_dataset,
        batch_size=512,
        shuffle=True,# shuffle is only 'True' for training sets.
        num_workers=4
)
' set up protein testing dataset with Dataset and Dataloader using pytorch'

test_protein_dataset = protein_sequences_dataset(
    data=test_Kd_df['protein_sequence'].tolist(),
    output_labels=test_Kd_df['log_norm_Kd'].tolist(),
    max_seq_len=max(len(s) for s in train_Kd_df['protein_sequence'].tolist())
)
test_protein_dataloader = DataLoader(
        test_protein_dataset,
        batch_size=512,
        shuffle=False, # shuffle is only 'True' for training sets.
        num_workers=4
)

In [ ]:
' here we will run a single pass through the data loader as an example '
train_x_proteins, train_y_true = next(iter(train_protein_dataloader))
print('Training set: Shape of the protein sequence input and output labels:', list(train_x_proteins.shape), list(train_y_true.shape))

' here we will run a single pass through the data loader as an example '
test_x_proteins, test_y_true = next(iter(test_protein_dataloader))
print('Testing set: Shape of the protein sequence input and output labels:', list(test_x_proteins.shape), list(test_y_true.shape))


In [ ]:
#@title Define the protein sequence-based model for affinity prediction

protein_seq_len = train_x_proteins.shape[1] # sequence length
protein_char_tokens = train_x_proteins.shape[-1] # number of protein sequence tokens
protein_input_size=protein_seq_len * protein_char_tokens # input dimension to MLP
protein_hidden_size=50 # hyperparameter; control expressive the neural network is...
protein_depth=1 # hyperparameter; control how deep the neural network is...

protein_model = simple_MLP(input_size=protein_input_size, hidden_size=protein_hidden_size, depth=protein_depth)
protein_model

## Note for training protein sequence-based encoder.

We can use the same training boiler plate, performance metric functions, and testing prediction functions as the ligand sequence-based encoder since our loss objective is the same. However, we will need to use our new dataloaders and model architecture for the protein dataset.

In [ ]:
#@title Train the Protein sequence-based model 🏋️‍♀️

#@markdown Here, since the training is quite fast for protein sequences unlike ligand SMILES strings, we will train the model from scratch.
#@markdown ---

# Define a loss function
criterion = nn.MSELoss()  # For simple regression prediction, we can use mean_squared_error loss.

# define an optimizer
optimizer = optim.SGD(protein_model.parameters(), lr=0.01)  # lr corresponds to learning_rate (keep fixed for now)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = protein_model.to(device)

#@markdown Decide whether to
train_model_option = 'train' #@param ['train', 'load_weights']{type:"string"}

# number of training cycles over the dataset (i.e. epochs)
num_epochs = 30 #@param [0, 1, 10, 15, 30] {type:"raw"}


# track losses and metrics
train_protein_history_dict = {
    'step': [],
    'loss': [],
    'pearson_corr': [],
    'spearman_rho': []
}

test_protein_history_dict = {
    'step': [],
    'loss': [],
    'pearson_corr': [],
    'spearman_rho': []
}

iteration = 0
# Start the training loop
for epoch in range(num_epochs):  # num_epochs is the number of epochs to train for
    for i, (x_protein, y_true) in enumerate(train_protein_dataloader ):
        # Move inputs and labels to the device
        x_protein, y_true = x_protein.to(device), y_true.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward propagation
        y_pred = protein_model(x_protein).squeeze(1)

        # Loss computation
        loss = criterion(y_pred, y_true)

        # Backpropagation
        loss.backward()

        # Weight update
        optimizer.step()
        pearson_corr, spearman_rho = performance_metrics(
                y_true=y_true,
                y_pred=y_pred
        )
        print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_protein_dataloader)}, Loss: {loss.item():.3f}, Pearson: {pearson_corr.item():.3f}, Spearman rho: {spearman_rho.item():.3f}")


        # log training performance
        train_protein_history_dict['step'].append(iteration)
        iteration+=1
        train_protein_history_dict['loss'].append(loss.item())
        train_protein_history_dict['pearson_corr'].append(pearson_corr.item())
        train_protein_history_dict['spearman_rho'].append(spearman_rho.item())

    # compute performance metrics on the test set
    test_loss, test_pearson_corr, test_spearman_rho = test_performance_metrics(model=protein_model, dataloader=test_protein_dataloader)
    test_protein_history_dict['step'].append(iteration)
    test_protein_history_dict['loss'].append(test_loss)
    test_protein_history_dict['pearson_corr'].append(test_pearson_corr)
    test_protein_history_dict['spearman_rho'].append(test_spearman_rho)



In [ ]:
#@title Visualize training history 🔎
fig, axes = plt.subplots(1, 3, figsize=(10,3))

axes[0].plot(train_protein_history_dict['step'], train_protein_history_dict['loss'])
axes[0].plot(test_protein_history_dict['step'], test_protein_history_dict['loss'], '.-', markersize=10)
axes[0].set_ylabel('loss')
axes[0].set_xlabel('steps')

axes[1].plot(train_protein_history_dict['step'], train_protein_history_dict['pearson_corr'])
axes[1].plot(test_protein_history_dict['step'], test_protein_history_dict['pearson_corr'], '.-', markersize=10)
axes[1].set_ylabel('Pearson corr.')
axes[1].set_xlabel('steps')

axes[2].plot(train_protein_history_dict['step'], train_protein_history_dict['spearman_rho'])
axes[2].plot(test_protein_history_dict['step'], test_protein_history_dict['spearman_rho'], '.-', markersize=10)
axes[2].set_ylabel('Spearman rho')
axes[2].set_xlabel('steps')

plt.tight_layout()

#### Summarize outputs:

We can see that the max sequence length for the protein sequence dataset is 1179, which is drastically longer than the SMILES strings. However, the number of class labels for representing the protein sequence (23) is lower than the SMILES string (35).

# Protein and Ligand sequence-based encoders model

The advantage of deep learning is the flexibility of handling multiple data types in one individual architecture. In other words, a deep learning architecture allows us to either take multiple inputs (e.g. protein sequence and ligand SMILES string) and output a single prediction (e.g. binding affinity) -- and vice-versa. This is usually referred to mutlimodal deep learning, and some resources in this category are the following:

  - short blog post: https://serokell.io/blog/multimodal-machine-learning
  - slides with many examples: https://web.stanford.edu/class/cs224n/slides/Multimodal-Deep-Learning-CS224n-Kiela.pdf
  - lecture for multimodal deep learning in protein engineering: https://www.youtube.com/watch?v=qFSVVWcCRHs
  - survey paper: https://dl.acm.org/doi/10.1109/TPAMI.2018.2798607
  
---
**Why is multimodal learning important for protein-ligand binding prediction?**

For starters, we have been predicting binding affinities using a neural network when it only has access to either protein sequence data or ligand data. This is of course is quite naive considering that binding affinites correspond to interaction between the protein sequence and ligand molecular and is not a intrinsic properties of the individual molecules. Thus, it makes more sense to develop a neural network model that predicts binding affinities given knowledge of the protein sequence and corresponding ligand graph. In addition, the preprocessing and representation of biomolcules like proteins and small molecular graphs like ligands are processed differently, and thus require a "multi-modal" approach.

🔥 **Goal** of this section is to develop a dataloader and train a neural network from scratch, which processes input data of both protein sequences and ligand SMILES strings and predicts regression value for $log \tilde{K_d}$. The architecture of this dual encoder is shown below.

In [ ]:
#@title Illustration of the Protein+Ligand-based sequence only encoder architecture.

#@markdown We will process the input data as Protein amino acid sequence data and use a simple MLP archiecture to predict real-value regression values $y$ (i.e. log( norm Kd)).

image_path = "./UChicago_AIplusScience_code_workshops/figures/Protein_Ligand_sequence_encoders.png"
display(Image(filename=image_path,width=500))

In [ ]:
#@title Remove cache and save memory.
import gc
# After deleting the DataLoader
del train_protein_dataloader, train_ligand_dataloader, test_protein_dataloader, test_ligand_dataloader
gc.collect()

In [ ]:
#@title Problem: create a dataloader that processes both Protein sequences and Ligand SMILES strings. 🚩

#@markdown Coding task 1 🚩: how would you fill in the following hook `preprocess_protein()` for processing the protein sequences? Recall: the input to this hook is strings containing amino acid characters, while the output should correspond to a padded torch tensor.

#@markdown Coding task 2 🚩: how would you fill in the following hook `preprocess_smiles()` for processing the ligand smiles? Recall: the input to this hook is strings containing SMILES characters, while the output should correspond to a padded torch tensor.

#@markdown Coding task 3 🚩: how would you fill in the `__getitem__()` which creates the batch samples to train your model? I left comments within the hook to indicate the three main steps to process protein and ligand sequence data in parallel: - (1) "preprocess protein data", "preprocess ligand data", and "preprocess output label data"


#@markdown note: we can use the dataloaders `protein_sequences_dataset` and `ligand_smiles_dataset` as a reference.


#@markdown ---

class protein_ligand_sequence_dataset(Dataset):

  def __init__(
      self,
      protein_data: list,
      protein_max_seq_len: int,
      ligand_data: list,
      ligand_max_seq_len: int,
      output_labels: list,
      ):

    # binding affinity measurements
    self.output_labels = output_labels

    ' preprocess protein data'
    self.protein_data = protein_data # input protein sequences
    self.protein_max_seq_len = protein_max_seq_len # max sequence length
    # Define the tokens to cover all the possible amino acids including special ones
    self.protein_tokens = [
        'A', 'C', 'D', 'E', 'F', 'G', 'H',
        'I', 'K', 'L', 'M', 'N', 'P', 'Q',
        'R', 'S', 'T', 'V', 'W', 'Y', 'U',
        'O', '-'
    ]
    # setup the amino acid tokenizer
    self.protein_tokenizer = {token:ii for ii, token in enumerate(self.protein_tokens)}

    ' preprocess ligand data'
    self.ligand_data = ligand_data # input ligand SMILES sequences
    self.ligand_max_seq_len = ligand_max_seq_len # max sequence length
    # ligand SMILEs tokenization
    self.ligand_tokenizer = dc.feat.OneHotFeaturizer() # ligand SMILEs tokenization

  def __len__(self):
    assert len(self.protein_data) == len(self.ligand_data), "Length of protein_data and ligand_data should be equal"
    return len(self.protein_data)

  def preprocess_protein(self, protein_sample):
      # convert amino acid characters into numerical values
      num_protein_sample = [self.protein_tokenizer[aa_char] for aa_char in protein_sample]
      # one hot encoding
      one_hot = F.one_hot(torch.tensor(num_protein_sample), num_classes=len(self.protein_tokens))
      # compute the padding length
      pad_len = self.protein_max_seq_len - one_hot.size(0)
      # create a tensor of zeros of shape (pad_len, len(self.tokens))
      pad_tensor = torch.zeros(pad_len, len(self.protein_tokens)).long()
      # set the padding tensor with the 'PAD' index
      pad_tensor[:, self.protein_tokenizer['-']] = 1
      # concatenate features and pad_tensor along dimension 0
      x_padded = torch.cat((one_hot, pad_tensor), dim=0)
      return x_padded

  def preprocess_smiles(self, smiles_sample):

    # convert string characters to one hot encoded tensors
    features =  torch.Tensor(self.ligand_tokenizer.featurize(smiles_sample)[:,0,:])
    # compute the padding length
    pad_len = self.ligand_max_seq_len - features.size(0)
    # create a tensor of zeros of shape (pad_len, 35)
    pad_tensor = torch.zeros(pad_len, 35)
    # set the last dimension of each vector in pad tensor to 1 (i.e. padded class)
    pad_tensor[:, -1] = 1
    # concatenate features and pad_tensor along dimension 0
    x_padded = torch.cat((features, pad_tensor), dim=0)
    return x_padded

  def __getitem__(self, idx):

    ' preprocess protein data'
    # prepare input data
    protein_sample = self.protein_data[idx]
    x_proteins = self.preprocess_protein(protein_sample=protein_sample)

    ' preprocess ligand data'
    # prepare input data
    ligands = self.ligand_data[idx]
    x_ligands = self.preprocess_smiles(smiles_sample=ligands)

    ' preprocess output label data'
    # prepare output/prediction data
    y_true = torch.tensor(self.output_labels[idx])

    return (
        x_proteins.float(),
        x_ligands.float(),
        y_true
    )





In [ ]:
' set up protein and ligand training dataset with Dataset and Dataloader using pytorch'
train_protein_ligand_dataset = protein_ligand_sequence_dataset(
    protein_data=train_Kd_df['protein_sequence'].tolist(),
    protein_max_seq_len=max(len(s) for s in train_Kd_df['protein_sequence'].tolist()),
    ligand_data=train_Kd_df['ligand_smiles'].tolist(),
    ligand_max_seq_len=max(len(s) for s in train_Kd_df['ligand_smiles'].tolist()),
    output_labels=train_Kd_df['log_norm_Kd'].tolist(),

)
train_protein_ligand_dataloader = DataLoader(
        train_protein_ligand_dataset,
        batch_size=512,
        shuffle=True,# shuffle is only 'True' for training sets.
        num_workers=0
)
' set up protein and ligand testing dataset with Dataset and Dataloader using pytorch'

test_protein_ligand_dataset = protein_ligand_sequence_dataset(
    protein_data=test_Kd_df['protein_sequence'].tolist(),
    protein_max_seq_len=max(len(s) for s in train_Kd_df['protein_sequence'].tolist()),
    ligand_data=test_Kd_df['ligand_smiles'].tolist(),
    ligand_max_seq_len=max(len(s) for s in train_Kd_df['ligand_smiles'].tolist()),
    output_labels=test_Kd_df['log_norm_Kd'].tolist(),
)
test_protein_ligand_dataloader = DataLoader(
        test_protein_ligand_dataset,
        batch_size=512,
        shuffle=False, # shuffle is only 'True' for training sets.
        num_workers=0
)

In [ ]:
' here we will run a single pass through the data loader as an example '
train_x_proteins, train_x_ligands, train_y_true = next(iter(train_protein_ligand_dataloader))
print('Training set: Shape of the protein sequence, ligand SMILES input and output labels:', train_x_proteins.shape, train_x_ligands.shape, train_y_true.shape)

' here we will run a single pass through the data loader as an example '
test_x_proteins, test_x_ligands, test_y_true = next(iter(test_protein_ligand_dataloader))
print('Testing set: Shape of the protein sequence, ligand SMILES input and output labels:', test_x_proteins.shape, test_x_ligands.shape, test_y_true.shape)


In [ ]:
#@title Problem: create a dual encoder MLP that each processes protein and ligand data 🚩

#@markdown `dual_encoder_MLP` is a neural network function that processes both Protein sequences and Ligand SMILES strings and predicts log $\tilde{K_d}$ values. The architecture is illustrated with the following figure.


#@markdown `Note:` Create a similar architecture as `simple_MLP` but have two encoders where each one processes either protein or ligand data.

#@markdown `Second Note:` Each encoder will convert the protein or ligand sequence into a hidden representation $h_s$ or $h_t$. We will concatenate these representations into $h=[h_s,h_t]$ and then process it through a downstream MLP head to predict binding affinities $log\tilde{K_d}$.

#@markdown ---

#@markdown Overall architecture should be the following:

#@markdown 1.) $h_p$ ~ $Encoder_{\theta_p}(x_p)$ and $h_l$ ~ $Encoder_{\theta_l}(x_l)$, where $x_p, x_l$ are the protein and ligand input data, $\theta_p$, $\theta_l$ are model parameters for protein sequences and ligand SMILES, and $h_p$, $h_l$ are hidden representations for protein and ligand sequences.

#@markdown 2.) h = [$h_p$, $h_l$]; concatenate the representations into an individual tensor

#@markdown 3.) y ~ $MLP_{\theta_b}(h)$, where $\theta_b$ is the model parameters for the downstream regression head and $y$ is the binding affinity output labels.

#@markdown ---

class dual_encoder_MLP(nn.Module):

  def __init__(
      self,
      protein_input_size: int,
      protein_hidden_size: int,
      protein_depth: int,
      ligand_input_size: int,
      ligand_hidden_size: int,
      ligand_depth: int,
      head_hidden_emb: int
    ):

    super(dual_encoder_MLP, self).__init__()

    # define hyperparameters
    self.protein_input_size = protein_input_size
    self.protein_hidden_size = protein_hidden_size
    self.protein_depth = protein_depth
    self.ligand_input_size = ligand_input_size
    self.ligand_hidden_size = ligand_hidden_size
    self.ligand_depth = ligand_depth
    self.head_hidden_emb = head_hidden_emb

    # define architecture layers
    self.protein_layers = self.protein_MLP() # h_p ~ Encoder(h_p | x_p)
    self.ligand_layers = self.ligand_MLP() # h_l ~ Encoder(h_l | x_l)
    self.head_layers = self.downstream_head_MLP() # y ~ MLP(y | h), where h = [h_p, h_l]


  def protein_MLP(self,):
      'note: similar architecture as simple_MLP without output layer'
      protein_layers = nn.ModuleList()

      # input layers
      protein_layers.append(nn.Linear(
          self.protein_input_size, self.protein_hidden_size
      ))
      protein_layers.append(nn.GELU())

      # hidden layers
      for _ in range(self.protein_depth - 1):
        protein_layers.append(nn.Linear(
                    self.protein_hidden_size, self.protein_hidden_size
        ))
        protein_layers.append(nn.GELU())

      return protein_layers

  def ligand_MLP(self,):
      'note: similar architecture as simple_MLP without output layer'
      ligand_layers = nn.ModuleList()

      # input layers
      ligand_layers.append(nn.Linear(
          self.ligand_input_size, self.ligand_hidden_size
      ))
      ligand_layers.append(nn.GELU())

      # hidden layers
      for _ in range(self.ligand_depth - 1):
        ligand_layers.append(nn.Linear(
                    self.ligand_hidden_size, self.ligand_hidden_size
        ))
        ligand_layers.append(nn.GELU())

      return ligand_layers

  def downstream_head_MLP(self,):
    'note: We will create the architecture for you, but you will need to figure out the right variables for the hyperparameters (e.g. input_dim)'

    input_dim = self.protein_hidden_size + self.ligand_hidden_size # remember h = [h_p, h_l]
    output_dim = 1 # regression task

    ' MLP head for regression'
    head_layers = nn.Sequential(
        nn.Linear(input_dim, self.head_hidden_emb),
        nn.GELU(),
        nn.Linear(self.head_hidden_emb, 1)
    )

    return head_layers

  # forward pass of neural network y ~ f(x)
  def forward(self, x_p, x_l):
    # Flatten the input tensor
    x_p = x_p.view(x_p.size(0), -1) # protein data
    x_l = x_l.view(x_l.size(0), -1) # ligand data

    # infer hidden representations for proteins
    h_p = x_p.clone()
    for layer in self.protein_layers:
      h_p = layer(h_p)

    # infer hidden representations for ligands
    h_l = x_l.clone()
    for layer in self.ligand_layers:
      h_l = layer(h_l)

    # concatenate representations
    h = torch.cat((h_p, h_l), dim=-1)

    # predict regression values
    y_pred = self.head_layers(h)

    return y_pred


In [ ]:
#@title Define the protein sequence-based model for affinity prediction

' Protein encoder hyperparameters '
protein_seq_len = train_x_proteins.shape[1] # sequence length
protein_char_tokens = train_x_proteins.shape[-1] # number of protein sequence tokens
protein_input_size=protein_seq_len * protein_char_tokens # input dimension to MLP
protein_hidden_size=500 # hyperparameter; control expressive the neural network is...
protein_depth=2 # hyperparameter; control how deep the neural network is...

' Ligand encoder hyperparameters '
ligand_seq_len = train_x_ligands.shape[1] # sequence length
ligand_char_tokens = train_x_ligands.shape[-1] # number of ligand sequence tokens
ligand_input_size=ligand_seq_len * ligand_char_tokens # input dimension to MLP
ligand_hidden_size=500 # hyperparameter; control expressive the neural network is...
ligand_depth=2 # hyperparameter; control how deep the neural network is...

' head hyperparameters'
head_hidden_emb=500 # hyperparameter; control expressive the neural network is..


# create model
protein_ligand_model = dual_encoder_MLP(
    protein_input_size=protein_input_size,
    protein_hidden_size=protein_hidden_size,
    protein_depth=protein_depth,
    ligand_input_size=ligand_input_size,
    ligand_hidden_size=ligand_hidden_size,
    ligand_depth=ligand_depth,
    head_hidden_emb=head_hidden_emb
)
protein_ligand_model

In [ ]:
#@title Function for compute metrics on test set for dual encoder 🛠️
@torch.no_grad()
def test_performance_metrics_on_dual(model: nn.Module, dataloader: DataLoader):

  model.eval()
  # Define a loss function
  criterion = nn.MSELoss()  # For simple regression prediction, we can use mean_squared_error loss.

  y_true_batch, y_pred_batch = [], []

  for i, (x_p, x_l, y_true) in enumerate(dataloader):

    # Move inputs and labels to the device
    x_p, x_l, y_true = x_p.to(device), x_l.to(device), y_true.to(device)

    # Forward propagation
    y_pred = model(x_p=x_p, x_l=x_l).squeeze(1)

    y_true_batch.append(y_true)
    y_pred_batch.append(y_pred)

  # concatenate batches
  y_true_batch = torch.cat(y_true_batch, dim=0)
  y_pred_batch = torch.cat(y_pred_batch, dim=0)

  # Loss computation
  loss = criterion(y_pred_batch, y_true_batch)

  # get metric performance
  pearson_corr, spearman_rho = performance_metrics(
                y_true=y_true_batch,
                y_pred=y_pred_batch
  )
  print(f"Test metrics | Loss: {loss.item():.3f}, Pearson: {pearson_corr.item():.3f}, Spearman rho: {spearman_rho.item():.3f}")

  return (
      loss.item(),
      pearson_corr.item(),
      spearman_rho.item()
  )



In [ ]:
#@title Train the Protein-ligand sequence-based model 🏋️‍♀️

#@title Train the ligand model

# Define a loss function
criterion = nn.MSELoss()  # For simple regression prediction, we can use mean_squared_error loss.

# define an optimizer
optimizer = optim.Adam(protein_ligand_model.parameters(), lr=0.0001)  # lr corresponds to learning_rate (keep fixed for now)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = protein_ligand_model.to(device)


#@markdown Since ligand-based encoder takes quite some time, we will load the weights of a pretrained model instead of train.


train_model_option = 'load_weights' #@param ['train', 'load_weights']{type:"string"}

# number of training cycles over the dataset (i.e. epochs)
num_epochs = 0 #@param [0, 1, 10, 15, 30] {type:"raw"}



# track losses and metrics
train_protein_ligand_history_dict = {
    'step': [],
    'loss': [],
    'pearson_corr': [],
    'spearman_rho': []
}

test_protein_ligand_history_dict = {
    'step': [],
    'loss': [],
    'pearson_corr': [],
    'spearman_rho': []
}

iteration = 0
# Start the training loop
for epoch in range(num_epochs):  # num_epochs is the number of epochs to train for
    for i, (x_protein, x_ligand, y_true) in enumerate(train_protein_ligand_dataloader ):
        # Move inputs and labels to the device
        x_protein, x_ligand, y_true = x_protein.to(device), x_ligand.to(device), y_true.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward propagation
        y_pred = protein_ligand_model(x_p=x_protein, x_l=x_ligand).squeeze(1)

        # Loss computation
        loss = criterion(y_pred, y_true)

        # Backpropagation
        loss.backward()

        # Weight update
        optimizer.step()
        pearson_corr, spearman_rho = performance_metrics(
                y_true=y_true,
                y_pred=y_pred
        )
        print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_protein_ligand_dataloader)}, Loss: {loss.item():.3f}, Pearson: {pearson_corr.item():.3f}, Spearman rho: {spearman_rho.item():.3f}")


        # log training performance
        train_protein_ligand_history_dict['step'].append(iteration)
        iteration+=1
        train_protein_ligand_history_dict['loss'].append(loss.item())
        train_protein_ligand_history_dict['pearson_corr'].append(pearson_corr.item())
        train_protein_ligand_history_dict['spearman_rho'].append(spearman_rho.item())

    # compute performance metrics on the test set
    test_loss, test_pearson_corr, test_spearman_rho = test_performance_metrics_on_dual(model=protein_ligand_model, dataloader=test_protein_ligand_dataloader)
    test_protein_ligand_history_dict['step'].append(iteration)
    test_protein_ligand_history_dict['loss'].append(test_loss)
    test_protein_ligand_history_dict['pearson_corr'].append(test_pearson_corr)
    test_protein_ligand_history_dict['spearman_rho'].append(test_spearman_rho)



In [ ]:
#@title Visualize training history 🔎

fig, axes = plt.subplots(1, 3, figsize=(10,3))

axes[0].plot(train_protein_ligand_history_dict['step'], train_protein_ligand_history_dict['loss'])
axes[0].plot(test_protein_ligand_history_dict['step'], test_protein_ligand_history_dict['loss'], '.-', markersize=10)
axes[0].set_ylabel('loss')
axes[0].set_xlabel('steps')

axes[1].plot(train_protein_ligand_history_dict['step'], train_protein_ligand_history_dict['pearson_corr'])
axes[1].plot(test_protein_ligand_history_dict['step'], test_protein_ligand_history_dict['pearson_corr'], '.-', markersize=10)
axes[1].set_ylabel('Pearson corr.')
axes[1].set_xlabel('steps')

axes[2].plot(train_protein_ligand_history_dict['step'], train_protein_ligand_history_dict['spearman_rho'])
axes[2].plot(test_protein_ligand_history_dict['step'], test_protein_ligand_history_dict['spearman_rho'], '.-', markersize=10)
axes[2].set_ylabel('Spearman rho')
axes[2].set_xlabel('steps')

plt.tight_layout()

In [ ]:
#@title #### Problem: can you use the loaded pretrain model to predict $log( norm(K_d))$ and compare against ground truth? 🚩

#@markdown Note: you can compute statistics like pearson correlation or spearmann $\rho$ correlation too.

In [ ]:
#@title Compare performance between the 3 approaches on the test set.

pretrained_p_seq_test_dict = torch.load('/content/UChicago_AIplusScience_code_workshops/pretrained_weights/test_Protein_sequence_history.pt')
pretrained_l_seq_test_dict = torch.load('/content/UChicago_AIplusScience_code_workshops/pretrained_weights/test_Ligand_sequence_history.pt')
pretrained_pl_seq_test_dict = torch.load('/content/UChicago_AIplusScience_code_workshops/pretrained_weights/test_Protein_Ligand_sequence_history.pt')


plt.bar(
    ['ligand encoder', 'protein encoder', 'protein-ligand \n dual encoder'],
    [pretrained_p_seq_test_dict['spearman_rho'][-1], pretrained_l_seq_test_dict['spearman_rho'][-1], pretrained_pl_seq_test_dict['spearman_rho'][-1]]
)
plt.ylabel('Spearmann rho on test set.')

# Problem: why did the dual encoder that takes both protein and ligand sequences performs better than the single encoder models? 🚩


## $<$Insert anwser here$>$

Note: think about what the actual problem...

## Sequence and structure-based encoder model

The next modeling arhcitecture will incorporate both the sequence-based information for both the protein and ligand sequence, while also introducing 3D structure data corresponding to the atom coordinates for the protein structure and ligand molecule. The hope is that the additional 3D data from the protein and ligand alongside the sequence information will improve binding affinity prediction on the test set. To process the 3D into representations, we will need to incorporate architectures that fall into the category of `Geometric deep learning`. This field has emerge as a exciting monolith in deep learning the last few yeasrs, with many successes for proteins and molecules (from protein structure prediction to molecule design). Here are some useful resources on geometric deep learing:

* blog: https://towardsdatascience.com/geometric-foundations-of-deep-learning-94cdd45b451d
* book: https://arxiv.org/abs/2104.13478
* video: https://www.youtube.com/watch?v=w6Pw4MOzMuo
    
   
Architectures for geometric deep learning can range from simple convolution neural networks to sophisicated graph neural networks. For our implementation and a simple introduce to working on 3D data, we will implement PointNet neural networks, which are models that effectively learn on pointclouds.

Here is an illustration that shows a 3D bed rendering that is converted into a pointcloud data type, which is then processed by a PointNet model and predicts the class label `bed`.


In [ ]:
#@title Point clouds allow to model 3D data.
from IPython.display import Image, display

image_path = "/content/UChicago_AIplusScience_code_workshops/figures/pointnet_video_illustration.gif"
display(Image(filename=image_path))

The exact details of the architecture of PointNet are not necessary for our current implementation; however, it is important to know that PointNet is permutation invariant which is well suited for working on cartesian coordinates. The main reasoning why PointNet achieves permutation invariance is thanks to the aggregated layer -- either a max pooling or average pooling operation. (run cell below). More details on PointNet can be found at the following links:

* Original paper: https://arxiv.org/abs/1612.00593
* Blog: http://stanford.edu/~rqi/pointnet/
* Notebook example from torch.geometric (Graph classification): https://colab.research.google.com/drive/1D45E5bUK3gQ40YpZo65ozs7hg5l-eo_U?usp=sharing

In [ ]:
#@title Global pooling layer for achieving permutation invariance.
image_path = "/content/UChicago_AIplusScience_code_workshops/figures/symmetric_function.PNG"
display(Image(filename=image_path))

`Goal:` Fe will need to create a dataset iterator that processes the protein sequences, ligand SMILES, and now both the protein and ligand atom coordinates as 3D data (i.e. point cloud).

`Note:` For this task, we will provide the architecture model and dataset iterator, but we will require for you to setup the forward pass hook for the PyTorch model.

`Second Note:` Working with 4 encoders and training a model on 3D data can be quite expensive, and thus we will provide a complete pretrained model. We will require you to create a function that infers and predicts binding affinities on test set. Then using the predicted measurements, we will ask you to compute performance metrics against the ground truth (e.g. $R^2$, etc).



In [ ]:
#@title Helpful functions to remove clutter from warning messages. 🛠️

import warnings
from Bio import BiopythonWarning

# Suppress specific warnings
warnings.filterwarnings("ignore", category=BiopythonWarning)


In [ ]:
#@title Create a dataloader that processes sequence and structure data. 🛠️

#@markdown `note:` We will provide `structure_sequence_dataset` dataset iterator for you since the object is quite sophisticated. However, please analyze the `__init__()` hook to understand what data variables are needed to process the dataset, and the `__getitem__()` hook to understand which data enters and exits object.


#@markdown ---

from Bio.PDB import PDBParser
from torch_geometric.data import Data

class structure_sequence_dataset(Dataset):

  def __init__(
      self,
      protein_seq_data: list,
      protein_max_seq_len: int,
      protein_structure_paths: list,
      ligand_seq_data: list,
      ligand_max_seq_len: int,
      ligand_structure_paths: list,
      output_labels: list,
      ):

    # binding affinity measurements
    self.output_labels = output_labels

    ' preprocess protein sequence data'
    self.protein_data = protein_seq_data # input protein sequences
    self.protein_max_seq_len = protein_max_seq_len # max sequence length
    # Define the tokens to cover all the possible amino acids including special ones
    self.protein_tokens = [
        'A', 'C', 'D', 'E', 'F', 'G', 'H',
        'I', 'K', 'L', 'M', 'N', 'P', 'Q',
        'R', 'S', 'T', 'V', 'W', 'Y', 'U',
        'O', '-'
    ]
    # setup the amino acid tokenizer
    self.protein_tokenizer = {token:ii for ii, token in enumerate(self.protein_tokens)}

    ' preprocess protein structure data'
    # process PDB structure
    self.parser = PDBParser() # comes from BioPython package (loads PDB structures)
    self.protein_paths = protein_structure_paths # structure paths

    ' preprocess ligand sequence data'
    self.ligand_data = ligand_seq_data # input ligand SMILES sequences
    self.ligand_max_seq_len = ligand_max_seq_len # max sequence length
    # ligand SMILEs tokenization
    self.ligand_tokenizer = dc.feat.OneHotFeaturizer() # ligand SMILEs tokenization

    ' preprocess ligand structure data'
    self.ligand_paths = ligand_structure_paths # structure paths


  def __len__(self):
    assert len(self.protein_data) == len(self.ligand_data), "Length of protein_data and ligand_data should be equal"
    return len(self.protein_data)

  def preprocess_protein_sequence(self, protein_sample):
      # convert amino acid characters into numerical values
      num_protein_sample = [self.protein_tokenizer[aa_char] for aa_char in protein_sample]
      # one hot encoding
      one_hot = F.one_hot(torch.tensor(num_protein_sample), num_classes=len(self.protein_tokens))
      # compute the padding length
      pad_len = self.protein_max_seq_len - one_hot.size(0)
      # create a tensor of zeros of shape (pad_len, len(self.tokens))
      pad_tensor = torch.zeros(pad_len, len(self.protein_tokens)).long()
      # set the padding tensor with the 'PAD' index
      pad_tensor[:, self.protein_tokenizer['-']] = 1
      # concatenate features and pad_tensor along dimension 0
      x_padded = torch.cat((one_hot, pad_tensor), dim=0)
      return x_padded

  def preprocess_smiles_sequence(self, smiles_sample):

    # convert string characters to one hot encoded tensors
    features =  torch.Tensor(self.ligand_tokenizer.featurize(smiles_sample)[:,0,:])
    # compute the padding length
    pad_len = self.ligand_max_seq_len - features.size(0)
    # create a tensor of zeros of shape (pad_len, 35)
    pad_tensor = torch.zeros(pad_len, 35)
    # set the last dimension of each vector in pad tensor to 1 (i.e. padded class)
    pad_tensor[:, -1] = 1
    # concatenate features and pad_tensor along dimension 0
    x_padded = torch.cat((features, pad_tensor), dim=0)
    return x_padded

  def pad_point_clouds(self, point_clouds, max_points):
        """
        we need to paddded 0s to a fixed size tensor for processing pytorch tools and models.
        """

        # pad the point clouds with a default value
        padded_point_clouds = []
        default_values = torch.tensor([0.0, 0.0, 0.0]) # default coordinates for the pads

        #for point_cloud in point_clouds:
        num_points = point_clouds.size(0)
        padding_size = max_points - num_points

        # pad point cloud
        padded_point_cloud = torch.cat([point_clouds, default_values.repeat(padding_size, 1)])

        # padded_point_clouds is now a list of point cloud tensors with the same number of points (max_points)
        return padded_point_cloud

  def preprocess_protein_structure(self, protein_path):

        structure = self.parser.get_structure("protein", protein_path)
        point_cloud = []

        # extract the atom coordinates
        for model in structure:
            for chain in model:
                for residue in chain:
                    for atom in residue:
                        if atom.get_name() == "CA": # consider only C-alpha atoms for backbone
                            coord = atom.get_coord()
                            point_cloud.append(coord)

        # assemble point cloud from atom coordinates as torch tensor
        point_cloud = torch.from_numpy(np.array(point_cloud)).float()

        # Center the point cloud to the center of mass
        center_of_mass = torch.mean(point_cloud, axis=0)
        centered_point_cloud = point_cloud - center_of_mass
        # pad point clouds to have save size tensors
        padded_point_cloud = self.pad_point_clouds(
            point_clouds=centered_point_cloud,
            max_points=5000
        )
        # note: torch.geometric Data object has several attributes, making it useful for us. We will only use 'pos' for point clouds.
        protein_structure_data = Data(pos=padded_point_cloud)

        # to access point cloud coordinates: protein_structure_data.pos
        return protein_structure_data



  def preprocess_ligand_structure(self, ligand_path):

        # load the molecule from the file
        mol = Chem.MolFromMol2File(ligand_path)


        if mol is None:
            # handle the case where molecule loading fails
            atom_coordinates = torch.zeros((0, 3), dtype=torch.float)

        else:
            try:
                # extract the ligand atoms coordinates
                atom_coordinates = torch.tensor(
                        mol.GetConformer().GetPositions()
                )
            except AttributeError:
                # Handle the case where conformer or positions are missing
                atom_coordinates = torch.zeros((0,3), dtype=torch.float)

         # Center the point cloud to the center of mass
        center_of_mass = torch.mean(atom_coordinates, axis=0)
        centered_point_cloud = atom_coordinates - center_of_mass

        # pad atom coordinates
        pad_atom_coordinates = self.pad_point_clouds(
            point_clouds=centered_point_cloud,
            max_points=500
        )
        # note: torch.geometric Data object has a several attributes, making it useful for us. We will only use 'pos' for point clouds.
        ligand_structure_data = Data(pos=pad_atom_coordinates)

        return ligand_structure_data


  def __getitem__(self, idx):

    ' preprocess protein sequence data'
    # prepare input data
    protein_sample = self.protein_data[idx]
    x_protein_seq = self.preprocess_protein_sequence(protein_sample=protein_sample)

    ' preprocess protein structure data'
    # prepare structure input data
    protein_paths = self.protein_paths[idx]
    x_protein_structures = self.preprocess_protein_structure(protein_path=protein_paths)
    x_protein_point_cloud = x_protein_structures.pos # extract point cloud

    ' preprocess ligand data'
    # prepare input data
    ligands = self.ligand_data[idx]
    x_ligand_seq = self.preprocess_smiles_sequence(smiles_sample=ligands)

    'preproces ligand structure data'
    ligand_paths = self.ligand_paths[idx]
    x_ligand_structures = self.preprocess_ligand_structure(ligand_path=ligand_paths)
    x_ligand_point_cloud = x_ligand_structures.pos # extract point cloud

    ' preprocess output label data'
    # prepare output/prediction data
    y_true = torch.tensor(self.output_labels[idx])

    return (
        x_protein_seq.float(),
        x_protein_point_cloud.float(),
        x_ligand_seq.float(),
        x_ligand_point_cloud.float(),
        y_true
    )


In [ ]:
#@title Reminder that we will need to use dataframe columns to define `structure_sequence_dataset` object

# here are the columns for the training set
display(train_Kd_df.head())

In [ ]:
#@title Problem: define the `structure_sequence_dataset` using the train_Kd_df and test_Kd_df dataframes  🚩

#@markdown `Note:` To understand which data variables to define `structure_sequence_dataset` iterator, please analyze `__init__()` inputs

#@markdown `Second Note:` Remember, we need data corresponding the protein sequence, ligand SMILES string, protein structure 3D data, and ligand 3D data.

#@markdown `Third Note:` To load structure data, we will need to leverage data paths and load structure per batch to avoid Out Of Memory issues (OOM). I.e., use columns `protein_pdb_paths` and `ligand_mol_paths`.

#@markdown ---

' set up sequence+structure training dataset with Dataset and Dataloader using pytorch'
train_seq_struct_dataset = structure_sequence_dataset(
    protein_seq_data=train_Kd_df['protein_sequence'].tolist(),
    protein_max_seq_len=max(len(s) for s in train_Kd_df['protein_sequence'].tolist()),
    protein_structure_paths=train_Kd_df['protein_pdb_paths'],
    ligand_seq_data=train_Kd_df['ligand_smiles'].tolist(),
    ligand_max_seq_len=max(len(s) for s in train_Kd_df['ligand_smiles'].tolist()),
    ligand_structure_paths=train_Kd_df['ligand_mol_paths'],
    output_labels=train_Kd_df['log_norm_Kd'].tolist(),
)
train_seq_struct_dataloader = DataLoader(
        train_seq_struct_dataset,
        batch_size=32,
        shuffle=True,# shuffle is only 'True' for training sets.
        num_workers=0
)
' set up sequence+structure testing dataset with Dataset and Dataloader using pytorch'

test_seq_struct_dataset = structure_sequence_dataset(
    protein_seq_data=test_Kd_df['protein_sequence'].tolist(),
    protein_max_seq_len=max(len(s) for s in train_Kd_df['protein_sequence'].tolist()),
    protein_structure_paths=test_Kd_df['protein_pdb_paths'],
    ligand_seq_data=test_Kd_df['ligand_smiles'].tolist(),
    ligand_max_seq_len=max(len(s) for s in train_Kd_df['ligand_smiles'].tolist()),
    ligand_structure_paths=train_Kd_df['ligand_mol_paths'],
    output_labels=test_Kd_df['log_norm_Kd'].tolist(),
)
test_seq_struct_dataloader = DataLoader(
        test_seq_struct_dataset,
        batch_size=32,
        shuffle=False, # shuffle is only 'True' for training sets.
        num_workers=0
)

In [ ]:
#@title Sanity check: example of running a single forward pass through the dataloaders. 🔎


' here we will run a single pass through the data loader as an example '
train_x_protein_seq, train_x_protein_cloud, train_x_ligand_seq, train_x_ligand_cloud, train_y_true = next(iter(train_seq_struct_dataloader))

print('Training set:')
print('='*50)
print('protein sequence shape (batch_size, sequence_length, class_labels) -> ', list(train_x_protein_seq.shape))
print('protein structure shape (batch_size, C-alpha_atoms, 3D_coordinates) -> ', list(train_x_protein_cloud.shape))
print('ligand sequence shape (batch_size, sequence_length, class_labels) -> ', list(train_x_ligand_seq.shape))
print('ligand structure shape (batch_size, atoms, 3D_coordinates) -> ', list(train_x_ligand_cloud.shape))





' here we will run a single pass through the data loader as an example '
test_x_protein_seq, test_x_protein_cloud, test_x_ligand_seq, test_x_ligand_cloud, test_y_true = next(iter(test_seq_struct_dataloader))

print('\nTesting set:')
print('='*50)
print('protein sequence shape (batch_size, sequence_length, class_labels) -> ', list(test_x_protein_seq.shape))
print('protein structure shape (batch_size, C-alpha_atoms, 3D_coordinates) -> ', list(test_x_protein_cloud.shape))
print('ligand sequence shape (batch_size, sequence_length, class_labels) -> ', list(test_x_ligand_seq.shape))
print('ligand structure shape (batch_size, atoms, 3D_coordinates) -> ', list(test_x_ligand_cloud.shape))



In [ ]:
#@title Functions for analyzing and visualizing point clouds. 🛠️🔎

import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interactive

def plot_point_cloud(angle):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")

    # Generate random point cloud data

    # Extract x, y, and z coordinates from the point cloud
    x = temp_point_cloud[:, 0]
    y = temp_point_cloud[:, 1]
    z = temp_point_cloud[:, 2]

    # Plot the point cloud
    ax.scatter(x, y, z, c="b", marker="o")

    # Set labels and title
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")
    ax.set_title("Point Cloud")

    # Rotate the plot
    ax.view_init(elev=angle, azim=angle)

    plt.show()


In [ ]:
#@title Visualize proteins as point clouds. 🔎


sample_index=4#@param{type:"integer"}
train_set=True#@param {type:"boolean"} # generate from the training

if train_set:
  temp_point_cloud = train_x_protein_cloud[sample_index]
  visualize_protein(pdb_path=train_Kd_df['protein_pdb_paths'].loc[sample_index])
  print(f"Binding affinity: {train_Kd_df['exp_measurement'].loc[sample_index]}")
else:
  temp_point_cloud = test_x_protein_cloud[sample_index]
  visualize_protein(pdb_path=test_Kd_df['protein_pdb_paths'].loc[sample_index])
  print(f"Binding affinity: {test_Kd_df['exp_measurement'].loc[sample_index]}")



# Create an interactive slider
angle_slider = interactive(
    plot_point_cloud,
    angle=(0, 360, 5)
)
display(angle_slider)

In [ ]:
#@title Visualize ligands as point clouds. 🔎



sample_index=12#@param{type:"integer"}
train_set=True#@param {type:"boolean"} # generate from the training

if train_set:
  temp_point_cloud = train_x_ligand_cloud[sample_index]
  visualize_ligand(ligand_sdf_path=train_Kd_df['ligand_sdf_paths'].loc[sample_index])
  print(f"Binding affinity: {train_Kd_df['exp_measurement'].loc[sample_index]}")
else:
  temp_point_cloud = test_x_ligand_cloud[sample_index]
  visualize_ligand(ligand_sdf_path=test_Kd_df['ligand_sdf_paths'].loc[sample_index])
  print(f"Binding affinity: {test_Kd_df['exp_measurement'].loc[sample_index]}")

# Create an interactive slider
angle_slider = interactive(
    plot_point_cloud,
    angle=(0, 360, 5)
)
display(angle_slider)


# Problem: Create a dual encoder MLP that each processes protein and ligand data.  🚩

Here, we will create and define the model architecture, but you will need to define the forward pass. In other words, you will need to setup which samples enter the model and which samples exit the model.

In [ ]:
#@title Functions for PointNet. 🛠️

#@markdown `note:` There is no need to understand all of the details of PointNet for brevity. It is important to understand that PointNet allows to process 3D data (e.g. PointCloud), but there is many alternative approaches like Graph Neural Networks, 3D Convolutional Neural Networks (voxelization).


#@markdown ---
import scipy.spatial

def knn(x, y, k, batch_x=None, batch_y=None):
    r"""Finds for each element in :obj:`y` the :obj:`k` nearest points in
    :obj:`x`.

    Args:
        x (Tensor): Node feature matrix
            :math:`\mathbf{X} \in \mathbb{R}^{N \times F}`.
        y (Tensor): Node feature matrix
            :math:`\mathbf{X} \in \mathbb{R}^{M \times F}`.
        k (int): The number of neighbors.
        batch_x (LongTensor, optional): Batch vector
            :math:`\mathbf{b} \in {\{ 0, \ldots, B-1\}}^N`, which assigns each
            node to a specific example. (default: :obj:`None`)
        batch_y (LongTensor, optional): Batch vector
            :math:`\mathbf{b} \in {\{ 0, \ldots, B-1\}}^M`, which assigns each
            node to a specific example. (default: :obj:`None`)

    :rtype: :class:`LongTensor`

    .. testsetup::

        import torch
        from torch_cluster import knn

    .. testcode::

        >>> x = torch.Tensor([[-1, -1], [-1, 1], [1, -1], [1, 1]])
        >>> batch_x = torch.tensor([0, 0, 0, 0])
        >>> y = torch.Tensor([[-1, 0], [1, 0]])
        >>> batch_x = torch.tensor([0, 0])
        >>> assign_index = knn(x, y, 2, batch_x, batch_y)
    """

    if batch_x is None:
        batch_x = x.new_zeros(x.size(0), dtype=torch.long)

    if batch_y is None:
        batch_y = y.new_zeros(y.size(0), dtype=torch.long)

    x = x.view(-1, 1) if x.dim() == 1 else x
    y = y.view(-1, 1) if y.dim() == 1 else y

    assert x.dim() == 2 and batch_x.dim() == 1
    assert y.dim() == 2 and batch_y.dim() == 1
    assert x.size(1) == y.size(1)
    assert x.size(0) == batch_x.size(0)
    assert y.size(0) == batch_y.size(0)

   # if x.is_cuda:
   #     return torch_cluster.knn_cuda.knn(x, y, k, batch_x, batch_y)

    # Rescale x and y.
    min_xy = min(x.min().item(), y.min().item())
    x, y = x - min_xy, y - min_xy

    max_xy = max(x.max().item(), y.max().item())
    x, y, = x / max_xy, y / max_xy

    # Concat batch/features to ensure no cross-links between examples exist.
    x = torch.cat([x, 2 * x.size(1) * batch_x.view(-1, 1).to(x.dtype)], dim=-1)
    y = torch.cat([y, 2 * y.size(1) * batch_y.view(-1, 1).to(y.dtype)], dim=-1)

    tree = scipy.spatial.cKDTree(x.detach().numpy())
    dist, col = tree.query(
        y.detach().cpu(), k=k, distance_upper_bound=x.size(1))
    dist = torch.from_numpy(dist).to(x.dtype)
    col = torch.from_numpy(col).to(torch.long)
    row = torch.arange(col.size(0), dtype=torch.long).view(-1, 1).repeat(1, k)

    mask = ~torch.isinf(dist).view(-1) # using the ~ operator
#    mask = 1 - torch.isinf(dist).view(-1)
    row, col = row.view(-1)[mask], col.view(-1)[mask]

    return torch.stack([row, col], dim=0)



def knn_graph(x, k, batch=None, loop=False, flow='source_to_target'):
    r"""Computes graph edges to the nearest :obj:`k` points.

    Args:
        x (Tensor): Node feature matrix
            :math:`\mathbf{X} \in \mathbb{R}^{N \times F}`.
        k (int): The number of neighbors.
        batch (LongTensor, optional): Batch vector
            :math:`\mathbf{b} \in {\{ 0, \ldots, B-1\}}^N`, which assigns each
            node to a specific example. (default: :obj:`None`)
        loop (bool, optional): If :obj:`True`, the graph will contain
            self-loops. (default: :obj:`False`)
        flow (string, optional): The flow direction when using in combination
            with message passing (:obj:`"source_to_target"` or
            :obj:`"target_to_source"`). (default: :obj:`"source_to_target"`)

    :rtype: :class:`LongTensor`

    .. testsetup::

        import torch
        from torch_cluster import knn_graph

    .. testcode::

        >>> x = torch.Tensor([[-1, -1], [-1, 1], [1, -1], [1, 1]])
        >>> batch = torch.tensor([0, 0, 0, 0])
        >>> edge_index = knn_graph(x, k=2, batch=batch, loop=False)
    """

    assert flow in ['source_to_target', 'target_to_source']
    row, col = knn(x, x, k if loop else k + 1, batch, batch)
    row, col = (col, row) if flow == 'source_to_target' else (row, col)
    if not loop:
        mask = row != col
        row, col = row[mask], col[mask]
    return torch.stack([row, col], dim=0)

In [ ]:
#@title Layers for PointNet. 🛠️

#@markdown `note:` There is no need to understand all of the details of PointNet for brevity. It is important to understand that PointNet allows to process 3D data (e.g. PointCloud), but there is many alternative approaches like Graph Neural Networks, 3D Convolutional Neural Networks (voxelization).

import torch.nn.functional as F
from torch_geometric.nn import global_max_pool
from sklearn.neighbors import NearestNeighbors
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing

class PointNetLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        # Message passing with "max" aggregation.
        super().__init__(aggr='max')

        # Initialization of the MLP:
        # Here, the number of input features correspond to the hidden node
        # dimensionality plus point dimensionality (=3).
        self.mlp = Sequential(Linear(in_channels + 3, out_channels),
                              ReLU(),
                              Linear(out_channels, out_channels))

    def forward(self, h, pos, edge_index):
        # Start propagating messages.
        return self.propagate(edge_index, h=h, pos=pos)

    def message(self, h_j, pos_j, pos_i):
        # h_j defines the features of neighboring nodes as shape [num_edges, in_channels]
        # pos_j defines the position of neighboring nodes as shape [num_edges, 3]
        # pos_i defines the position of central nodes as shape [num_edges, 3]

        input = pos_j - pos_i  # Compute spatial relation.

        if h_j is not None:
            # In the first layer, we may not have any hidden node features,
            # so we only combine them in case they are present.
            input = torch.cat([h_j, input], dim=-1)

        return self.mlp(input)  # Apply our final MLP.


class PointNet(torch.nn.Module):
    def __init__(self, emb_dim: int=32):
        super().__init__()

        torch.manual_seed(42)
        self.emb_dim = emb_dim
        self.num_classes = 1
        self.conv1 = PointNetLayer(3, emb_dim)
        self.conv2 = PointNetLayer(emb_dim, emb_dim)

    def forward(self, pos, batch):



        # Compute the kNN graph:
        # Here, we need to pass the batch vector to the function call in order
        # to prevent creating edges between points of different examples.
        edge_index = knn_graph(pos.cpu(), k=16, batch=batch.cpu(), loop=True).to(pos.device)
        batch = batch.to(pos.device).long()

        # 3. Start bipartite message passing.
        h = self.conv1(h=pos, pos=pos, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(h=h, pos=pos, edge_index=edge_index)
        h = h.relu()

        # 4. Global Pooling.
        h = global_max_pool(h, batch=batch)

        # representations h
        return h


In [ ]:
#@title Create a quad. encoder net that processes protein and ligand seq+structure data 🛠️

#@markdown `quad_encoder_net` is a neural network function that processes both Protein sequences and Ligand SMILES strings + protein and ligand point clouds and predicts log $\tilde{K_d}$ values. The architecture is illustrated with the following figure.

#@markdown `Note:` Create a similar architecture as `simple_MLP` for the two seuence encoders. We will introduce two more structure encodes using PointNet model.

#@markdown `Second Note:` Each sequence encoder will convert the protein and ligand sequence into a hidden representation $h_s$ or $h_t$. Each structure encoder (i.e. PointNet) will convert the protein and ligand structure into a hidden representation $H_p$ and $H_l$. We will concatenate these representations into $H=[h_p,H_p,h_l,H_l]$ and then process it through a downstream MLP head to predict binding affinities $log\tilde{K_d}$.

#@markdown ---

#@markdown Overall architecture should be the following:

#@markdown 1.) $h_p$ ~ $Encoder_{\theta_p}(x_p)$, $H_p$ ~ $Encoder_{\omega_p}(X_p)$, $h_l$ ~ $Encoder_{\theta_l}(x_l)$, $H_l$ ~ $Encoder_{\omega_l}(X_l)$, where $x_p, X_p, x_l, X_l$ are the protein sequence, protein structure, ligand sequence, and ligand structure input data, $\theta_p$, $\omega_p$, $\theta_l$, $\omega_l$ are model parameters for protein sequences, protein structures, ligand SMILES strings, and ligand SMILES structures. The $h_p$, $H_p$, $h_l$, and $H_l$ are hidden representations for protein sequences, protein structures, ligand sequences, and ligand structures.

#@markdown 2.) H = [$h_p$, $H_p$, $h_l$, $H_l$]; concatenate the representations into an individual tensor

#@markdown 3.) y ~ $MLP_{\theta_b}(H)$, where $\theta_b$ is the model parameters for the downstream regression head and $y$ is the binding affinity output labels.


#@markdown `note:` Please analyze the `__init__()` to understand the variables required to assemble the model and `forward()` to understand the input data that is required to predict binding affinities.

#@markdown ---



class quad_encoder_net(nn.Module):

  def __init__(
      self,
      protein_input_size: int,
      protein_hidden_size: int,
      protein_depth: int,
      ligand_input_size: int,
      ligand_hidden_size: int,
      ligand_depth: int,
      pointnet_emb_dim: int,
      head_hidden_emb: int
    ):

    super(quad_encoder_net, self).__init__()

    # define hyperparameters
    self.protein_input_size = protein_input_size
    self.protein_hidden_size = protein_hidden_size
    self.protein_depth = protein_depth
    self.ligand_input_size = ligand_input_size
    self.ligand_hidden_size = ligand_hidden_size
    self.ligand_depth = ligand_depth
    self.pointnet_emb_dim = pointnet_emb_dim
    self.head_hidden_emb = head_hidden_emb

    # define architecture layers
    ' protein encoders'
    self.protein_seq_layers = self.protein_MLP() # h_p ~ Encoder(h_p | x_p)
    self.protein_struct_net = PointNet(emb_dim=self.pointnet_emb_dim)

    ' ligand encoders'
    self.ligand_seq_layers = self.ligand_MLP() # h_l ~ Encoder(h_l | x_l)
    self.ligand_struct_net = PointNet(emb_dim=self.pointnet_emb_dim)

    ' top regression head'
    self.head_layers = self.downstream_head_MLP() # y ~ MLP(y | h), where h = [h_p, h_l]


  def protein_MLP(self,):
      'note: similar architecture as simple_MLP without output layer'
      protein_layers = nn.ModuleList()

      # input layers
      protein_layers.append(nn.Linear(
          self.protein_input_size, self.protein_hidden_size
      ))
      protein_layers.append(nn.GELU())

      # hidden layers
      for _ in range(self.protein_depth - 1):
        protein_layers.append(nn.Linear(
                    self.protein_hidden_size, self.protein_hidden_size
        ))
        protein_layers.append(nn.GELU())

      return protein_layers

  def ligand_MLP(self,):
      'note: similar architecture as simple_MLP without output layer'
      ligand_layers = nn.ModuleList()

      # input layers
      ligand_layers.append(nn.Linear(
          self.ligand_input_size, self.ligand_hidden_size
      ))
      ligand_layers.append(nn.GELU())

      # hidden layers
      for _ in range(self.ligand_depth - 1):
        ligand_layers.append(nn.Linear(
                    self.ligand_hidden_size, self.ligand_hidden_size
        ))
        ligand_layers.append(nn.GELU())

      return ligand_layers

  def downstream_head_MLP(self,):
    'note: We will create the architecture for you, but you will need to figure out the right variables for the hyperparameters (e.g. input_dim)'

    tot_point_emb_dim = 2*(self.pointnet_emb_dim)
    input_dim = tot_point_emb_dim + self.protein_hidden_size + self.ligand_hidden_size # remember h = [h_p, h_l]
    output_dim = 1 # regression task

    ' MLP head for regression'
    head_layers = nn.Sequential(
        nn.Linear(input_dim, self.head_hidden_emb),
        nn.GELU(),
        nn.Linear(self.head_hidden_emb, 1)
    )

    return head_layers

  # forward pass of neural network y ~ f(x)
  def forward(self, x_seq_p, x_struct_p, x_seq_l, x_struct_l):

    # Flatten the input tensor
    x_seq_p = x_seq_p.view(x_seq_p.size(0), -1) # protein sequence data
    x_seq_l = x_seq_l.view(x_seq_l.size(0), -1) # ligand sequence data

    ' infer hidden representations for proteins sequences '
    h_seq_p = x_seq_p.clone()
    for layer in self.protein_seq_layers:
        h_seq_p = layer(h_seq_p)

    ' infer hidden representations for proteins structure '
    h_struct_p, batch_size, num_nodes = [], x_struct_p.shape[0], x_struct_p.shape[1]
    # Create the batch vector with cyclic pattern
    batch = torch.arange(batch_size).repeat_interleave(num_nodes)
    # compute structure hidden representation
    h_struct_p = self.protein_struct_net(x_struct_p.view(-1, 3), batch)

    ' infer hidden representations for ligands sequences '
    h_seq_l = x_seq_l.clone()
    for layer in self.ligand_seq_layers:
        h_seq_l = layer(h_seq_l)

    ' infer hidden representations for ligand structure '
    h_struct_l, batch_size, num_nodes = [], x_struct_l.shape[0], x_struct_l.shape[1]
    # create the batch vector with cyclic pattern
    batch = torch.arange(batch_size).repeat_interleave(num_nodes)
    # compute structure hidden representation
    h_struct_l = self.ligand_struct_net(x_struct_l.view(-1, 3), batch)

    # concatenate representations
    h = torch.cat((h_seq_p, h_struct_p, h_seq_l, h_struct_l), dim=-1)


    ' predict regression values '
    y_pred = self.head_layers(h)

    return y_pred


In [ ]:
#@title Problem: define the protein sequence-based model for affinity prediction  🚩


'point net structure hyperparameters for both ligand and protein encoders'
pointnet_emb_dim = 32 # pointnet hyperparameter; control expressiveness

' Protein encoder hyperparameters '
protein_seq_len = train_x_protein_seq.shape[1] # sequence length
protein_char_tokens = train_x_protein_seq.shape[-1] # number of protein sequence tokens
protein_input_size=protein_seq_len * protein_char_tokens # input dimension to MLP
protein_hidden_size=500 # hyperparameter; control expressive the neural network is...
protein_depth=2 # hyperparameter; control how deep the neural network is...

' Ligand encoder hyperparameters '
ligand_seq_len = train_x_ligand_seq.shape[1] # sequence length
ligand_char_tokens = train_x_ligand_seq.shape[-1] # number of ligand sequence tokens
ligand_input_size=ligand_seq_len * ligand_char_tokens # input dimension to MLP
ligand_hidden_size=500 # hyperparameter; control expressiveness of the neural network is...
ligand_depth=2 # hyperparameter; control how deep the neural network is...

' head hyperparameters'
head_hidden_emb=2000 # hyperparameter; control expressive the neural network is..


# Problem: create model by filling in the hyperparameters
BioMol_seq_struct_model = quad_encoder_net(
    protein_input_size=protein_input_size,
    protein_hidden_size=protein_hidden_size,
    protein_depth=protein_depth,
    ligand_input_size=ligand_input_size,
    ligand_hidden_size=ligand_hidden_size,
    ligand_depth=ligand_depth,
    pointnet_emb_dim=pointnet_emb_dim,
    head_hidden_emb=head_hidden_emb,
)


In [ ]:

#@title Function for compute metrics on test set for quad. encoder 🛠️
@torch.no_grad()
def test_performance_metrics_on_quad(model: nn.Module, dataloader: DataLoader):


    model.eval()
    # Define a loss function
    criterion = nn.MSELoss()  # For simple regression prediction, we can use mean_squared_error loss.

    y_true_batch, y_pred_batch = [], []

    for i, (x_seq_p, x_struct_p, x_seq_l, x_struct_l, y_true) in enumerate(dataloader):

        # Move inputs and labels to the device
        x_seq_p, x_struct_p, x_seq_l, x_struct_l, y_true = x_seq_p.to(device), x_struct_p.to(device), x_seq_l.to(device), x_struct_l.to(device), y_true.to(device)


        # Forward propagation
        y_pred = model(
            x_seq_p=x_seq_p,
            x_struct_p=x_struct_p,
            x_seq_l=x_seq_l,
            x_struct_l=x_struct_l
        ).squeeze(1)

        y_true_batch.append(y_true)
        y_pred_batch.append(y_pred)

        # concatenate batches
        y_true_batch = torch.cat(y_true_batch, dim=0)
        y_pred_batch = torch.cat(y_pred_batch, dim=0)

        # Loss computation
        loss = criterion(y_pred_batch, y_true_batch)

        # get metric performance
        pearson_corr, spearman_rho = performance_metrics(
                    y_true=y_true_batch,
                    y_pred=y_pred_batch
        )
        print(f"Test metrics | Loss: {loss.item():.3f}, Pearson: {pearson_corr.item():.3f}, Spearman rho: {spearman_rho.item():.3f}")


        return (
          loss.item(),
          pearson_corr.item(),
          spearman_rho.item()
        )



In [ ]:
#@title Train the Sequence-Structure model 🏋️‍♀️

# Define a loss function
criterion = nn.MSELoss()  # For simple regression prediction, we can use mean_squared_error loss.

# define an optimizer
optimizer = optim.Adam(BioMol_seq_struct_model.parameters(), lr=0.0001)  # lr corresponds to learning_rate (keep fixed for now)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BioMol_seq_struct_model = BioMol_seq_struct_model.to(device)

# number of training cycles over the dataset (i.e. epochs)
num_epochs=1#@param{type:"integer"}


# track losses and metrics
train_seq_struct_history_dict = {
    'step': [],
    'loss': [],
    'pearson_corr': [],
    'spearman_rho': []
}

test_seq_struct_history_dict = {
    'step': [],
    'loss': [],
    'pearson_corr': [],
    'spearman_rho': []
}

iteration = 0
# Start the training loop
for epoch in range(num_epochs):  # num_epochs is the number of epochs to train for
    for i, (x_seq_p, x_struct_p, x_seq_l, x_struct_l, y_true) in enumerate(train_seq_struct_dataloader ):
        # Move inputs and labels to the device
        x_seq_p, x_struct_p, x_seq_l, x_struct_l, y_true = x_seq_p.to(device), x_struct_p.to(device), x_seq_l.to(device), x_struct_l.to(device), y_true.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward propagation
        y_pred = BioMol_seq_struct_model(
            x_seq_p=x_seq_p,
            x_struct_p=x_struct_p,
            x_seq_l=x_seq_l,
            x_struct_l=x_struct_l,
        ).squeeze(1)

        # Loss computation
        loss = criterion(y_pred, y_true)

        # Backpropagation
        loss.backward()

        # Weight update
        optimizer.step()
        pearson_corr, spearman_rho = performance_metrics(
                y_true=y_true,
                y_pred=y_pred
        )
        print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_seq_struct_dataloader)}, Loss: {loss.item():.3f}, Pearson: {pearson_corr.item():.3f}, Spearman rho: {spearman_rho.item():.3f}")


        # log training performance
        train_seq_struct_history_dict['step'].append(iteration)
        iteration+=1
        train_seq_struct_history_dict['loss'].append(loss.item())
        train_seq_struct_history_dict['pearson_corr'].append(pearson_corr.item())
        train_seq_struct_history_dict['spearman_rho'].append(spearman_rho.item())

    # compute performance metrics on the test set
    test_loss, test_pearson_corr, test_spearman_rho = test_performance_metrics_on_quad(model=BioMol_seq_struct_model, dataloader=test_seq_struct_dataloader)
    test_seq_struct_history_dict['step'].append(iteration)
    test_seq_struct_history_dict['loss'].append(test_loss)
    test_seq_struct_history_dict['pearson_corr'].append(test_pearson_corr)
    test_seq_struct_history_dict['spearman_rho'].append(test_spearman_rho)



In [ ]:
#@title Plot training + testing history

fig, axes = plt.subplots(1, 3, figsize=(10,3))

axes[0].plot(train_seq_struct_history_dict['step'], train_seq_struct_history_dict['loss'])
axes[0].plot(test_seq_struct_history_dict['step'], test_seq_struct_history_dict['loss'], '.-', markersize=10)
axes[0].set_ylabel('loss')
axes[0].set_xlabel('steps')

axes[1].plot(train_seq_struct_history_dict['step'], train_seq_struct_history_dict['pearson_corr'])
axes[1].plot(test_seq_struct_history_dict['step'], test_seq_struct_history_dict['pearson_corr'], '.-', markersize=10)
axes[1].set_ylabel('Pearson corr.')
axes[1].set_xlabel('steps')

axes[2].plot(train_seq_struct_history_dict['step'], train_seq_struct_history_dict['spearman_rho'])
axes[2].plot(test_seq_struct_history_dict['step'], test_seq_struct_history_dict['spearman_rho'], '.-', markersize=10)
axes[2].set_ylabel('Spearman rho')
axes[2].set_xlabel('steps')

plt.tight_layout()

In [ ]:
#@title Compare performance between the 3 approaches on the test set.

#pretrained_p_seq_test_dict = torch.load('/content/UChicago_AIplusScience_code_workshops/pretrained_weights/test_Protein_sequence_history.pt')
#pretrained_l_seq_test_dict = torch.load('/content/UChicago_AIplusScience_code_workshops/pretrained_weights/test_Ligand_sequence_history.pt')
#pretrained_pl_seq_test_dict = torch.load('/content/UChicago_AIplusScience_code_workshops/pretrained_weights/test_Protein_Ligand_sequence_history.pt')
pretrained_pl_seq_struct_test_dict = torch.load('/content/UChicago_AIplusScience_code_workshops/pretrained_weights/test_Sequene_Structure_history.pt')


plt.bar(
    ['ligand encoder',
     'protein encoder',
     'protein-ligand \n dual encoder',
     'protein-ligand \n seq-struct. encoders'],
    [
        pretrained_p_seq_test_dict['spearman_rho'][-1],
        pretrained_l_seq_test_dict['spearman_rho'][-1],
        pretrained_pl_seq_test_dict['spearman_rho'][-1],
        pretrained_pl_seq_struct_test_dict['spearman_rho'][-1]
        ]
)
plt.ylabel('Spearmann rho on test set.')

plt.tight_layout()

## Extension examples:
- docking then use point cloud representations
- finding new protein-ligand through your predictor.
- soph. architectures (e.g. graph neural networks for structure or transformers for sequence).